## Setup

In [47]:
from IPython.display import display, HTML 
display(HTML("<style>.container { width:80% !important; }</style>"))

In [1]:
import dbdreader
import gsw

import numpy as np
import pandas as pd

from datetime import timedelta, datetime

import math
import re

import sqlite3

from gpxcsv import gpxtolist

import requests

import plotly.graph_objects as go

from dash import Dash, dcc, html, Input, Output, State
import dash_ag_grid as dag

In [41]:
# For use in format: dbdreader.MultiDBD(pattern=DDIR+FILE+f'*.{file_format}', cacheDir=CDIR)
DDIR = '../Example Glider Data/Binaries/Koskelo/'
FILE = '' #'koskelo-2023-313-3-8'
CDIR = '../Example Glider Data/Caches/Koskelo/'
file_format = '[de]bd'

In [42]:
xbd = dbdreader.MultiDBD(pattern=DDIR+FILE+f'*.{file_format}', cacheDir=CDIR)
#xbd.set_time_limits("9 Nov 2023 00:00", "11 Nov 2023 00:00")

In [43]:
t,var=xbd.get("m_present_time")
pd.to_datetime(t, unit='s').sort_values()

DatetimeIndex([   '2023-11-09 07:07:55.036000',
               '2023-11-09 07:07:59.043000064',
               '2023-11-09 07:08:03.045000192',
               '2023-11-09 07:08:07.046999808',
               '2023-11-09 07:08:11.048999936',
               '2023-11-09 07:11:19.157999872',
               '2023-11-09 07:11:23.165999872',
               '2023-11-09 07:11:27.167000064',
               '2023-11-09 07:11:31.168999936',
                  '2023-11-09 07:11:35.172000',
               ...
               '2023-11-10 23:56:05.351000064',
               '2023-11-10 23:56:09.354000128',
               '2023-11-10 23:56:13.357000192',
               '2023-11-10 23:56:17.359000064',
               '2023-11-10 23:56:21.360999936',
                  '2023-11-10 23:56:25.364000',
               '2023-11-10 23:56:29.365999872',
               '2023-11-10 23:56:33.368999936',
                  '2023-11-10 23:56:37.372000',
               '2023-11-10 23:56:41.374000128'],
              dtype=

In [52]:
xbd.parameterNames['sci']+xbd.parameterNames['eng']

['sci_ad2cp_file_state',
 'sci_badd_error',
 'sci_badd_finished',
 'sci_badd_n_tries_to_connect',
 'sci_badd_power_on',
 'sci_badd_target_range',
 'sci_clothesline_time',
 'sci_ctd41cp_is_installed',
 'sci_ctd41cp_timestamp',
 'sci_generic_a',
 'sci_generic_b',
 'sci_generic_c',
 'sci_generic_d',
 'sci_generic_e',
 'sci_generic_f',
 'sci_generic_g',
 'sci_generic_h',
 'sci_generic_i',
 'sci_generic_j',
 'sci_generic_k',
 'sci_generic_l',
 'sci_generic_m',
 'sci_generic_n',
 'sci_generic_o',
 'sci_generic_p',
 'sci_generic_q',
 'sci_generic_r',
 'sci_generic_s',
 'sci_generic_t',
 'sci_generic_u',
 'sci_generic_v',
 'sci_generic_w',
 'sci_generic_x',
 'sci_generic_y',
 'sci_generic_z',
 'sci_log_time',
 'sci_m_disk_free',
 'sci_m_disk_usage',
 'sci_m_free_heap',
 'sci_m_min_free_heap',
 'sci_m_present_secs_into_mission',
 'sci_m_present_time',
 'sci_m_science_on',
 'sci_m_sram_free_heap',
 'sci_m_sram_min_free_heap',
 'sci_microrider_isdp_file_state',
 'sci_oxy4_c1amp',
 'sci_oxy4_c1rph

## EDA

### Uniqueness

In [53]:
noisy_variables = ["m_altimeter_status", "m_digifin_status", "m_fin", "m_is_ballast_pump_moving", "m_is_battpos_moving"]
# noisy_variables = ["m_ballast_pumped", "m_battpos", "m_fin"]

for var_name in noisy_variables:
    t,var=xbd.get(var_name)
    print(var_name + " " + str(len(var)) + " obs, " + str(len(np.unique(var))) + " unique")

m_altimeter_status 22540 obs, 9 unique
m_digifin_status 32352 obs, 4 unique
m_fin 32352 obs, 503 unique
m_is_ballast_pump_moving 32352 obs, 2 unique
m_is_battpos_moving 32352 obs, 2 unique


In [54]:
categoricals = ["m_altimeter_status", "m_digifin_status", "m_is_ballast_pump_moving", "m_is_battpos_moving"]

for var_name in categoricals:
    t,var=xbd.get(var_name)
    print(var_name + " " + str(len(var)) + " obs, taking values: " + ', '.join([str(i) for i in np.unique(var)]))

m_altimeter_status 22540 obs, taking values: 0.0, 1.0, 2.0, 3.0, 4.0, 6.0, 7.0, 10.0, 12.0
m_digifin_status 32352 obs, taking values: 0.0, 2.0, 256.0, 258.0
m_is_ballast_pump_moving 32352 obs, taking values: 0.0, 1.0
m_is_battpos_moving 32352 obs, taking values: 0.0, 1.0


We can't use get_sync or get_CTD_sync since they linearly interpolate categorical variables:

In [55]:
tctd, C, T, P, var = xbd.get_CTD_sync("m_is_ballast_pump_moving")
np.unique(var)

array([0.00000000e+00, 4.99773467e-04, 7.49443159e-04, ...,
       9.99250557e-01, 9.99500286e-01, 1.00000000e+00])

In [56]:
len(tctd)

120265

### Locating NaNs

#### CTD timebase

We use CTD timestamps as the base and find this approach doesn't work:

In [57]:
tctd, C, T, P = xbd.get_CTD_sync()

In [58]:
len(tctd)

120268

In [59]:
df = pd.DataFrame()
df["time"] = tctd
for var_name in noisy_variables:
    t,var=xbd.get(var_name)
    var_df = pd.DataFrame({"time":t, var_name:var})
    df = df.merge(var_df, how='outer', on="time")
df["time"] = pd.to_datetime(df["time"], unit='s')
df.sort_values(by=["time"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...
152580,2023-11-10 23:56:40.548000000,NaN,NaN,NaN,NaN,NaN
152581,2023-11-10 23:56:41.374000128,2.0,258.0,-0.002895,0.0,0.0
152582,2023-11-10 23:56:41.548000000,NaN,NaN,NaN,NaN,NaN
152583,2023-11-10 23:56:42.548000000,NaN,NaN,NaN,NaN,NaN


In [60]:
df[~df["time"].isin(pd.to_datetime(tctd, unit='s'))]

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...
152561,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0
152566,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0
152571,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0
152576,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0


In [61]:
df[df["time"].isin(pd.to_datetime(tctd, unit='s'))]

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving
2409,2023-11-09 11:45:11.204000000,NaN,NaN,NaN,NaN,NaN
2410,2023-11-09 11:45:11.897000192,NaN,NaN,NaN,NaN,NaN
2411,2023-11-09 11:45:12.897000192,NaN,NaN,NaN,NaN,NaN
2412,2023-11-09 11:45:13.898999808,NaN,NaN,NaN,NaN,NaN
2413,2023-11-09 11:45:14.897000192,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
152579,2023-11-10 23:56:39.548000000,NaN,NaN,NaN,NaN,NaN
152580,2023-11-10 23:56:40.548000000,NaN,NaN,NaN,NaN,NaN
152582,2023-11-10 23:56:41.548000000,NaN,NaN,NaN,NaN,NaN
152583,2023-11-10 23:56:42.548000000,NaN,NaN,NaN,NaN,NaN


In [62]:
df[df["time"].isin(pd.to_datetime(tctd, unit='s')) & (df["m_altimeter_status"].notna() | 
                                                      df["m_digifin_status"].notna() | 
                                                      df["m_fin"].notna() | 
                                                      df["m_is_ballast_pump_moving"].notna() | 
                                                      df["m_is_battpos_moving"].notna())]

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving
7800,2023-11-09 13:05:21.032999936,2.0,0.0,-0.008685,0.0,0.0
8434,2023-11-09 13:46:12.453999872,2.0,0.0,-0.020264,0.0,0.0
9354,2023-11-09 13:58:29.453000192,2.0,0.0,-0.057897,0.0,0.0
17632,2023-11-09 16:00:18.263000064,NaN,0.0,-0.144742,0.0,0.0
23906,2023-11-09 17:32:15.620999936,2.0,0.0,-0.101319,0.0,0.0
28445,2023-11-09 18:40:34.358000128,2.0,0.0,-0.214218,0.0,0.0
34876,2023-11-09 20:14:08.397000192,2.0,2.0,0.005790,0.0,0.0
40473,2023-11-09 21:28:55.516999936,2.0,2.0,0.296301,0.0,0.0
44545,2023-11-09 22:23:19.860999936,12.0,2.0,0.086845,0.0,0.0
46246,2023-11-09 22:46:01.859000064,2.0,2.0,-0.228693,0.0,0.0


Almost all the measurements are in a different timebase from CTD and instead share their timebase with each other.

They're also partially outside CTDs timeframe, meaning we can't simply use the CTD timeframe.

In [63]:
df[7795:7806]

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving
7795,2023-11-09 13:05:17.029999872,2.0,0.0,-0.008685,0.0,0.0
7796,2023-11-09 13:05:17.032000000,NaN,NaN,NaN,NaN,NaN
7797,2023-11-09 13:05:18.032999936,NaN,NaN,NaN,NaN,NaN
7798,2023-11-09 13:05:19.032999936,NaN,NaN,NaN,NaN,NaN
7799,2023-11-09 13:05:20.032999936,NaN,NaN,NaN,NaN,NaN
7800,2023-11-09 13:05:21.032999936,2.0,0.0,-0.008685,0.0,0.0
7801,2023-11-09 13:05:22.032999936,NaN,NaN,NaN,NaN,NaN
7802,2023-11-09 13:05:23.032999936,NaN,NaN,NaN,NaN,NaN
7803,2023-11-09 13:05:24.032999936,NaN,NaN,NaN,NaN,NaN
7804,2023-11-09 13:05:25.032999936,NaN,NaN,NaN,NaN,NaN


In [64]:
for var_name in noisy_variables:
    t,var=xbd.get(var_name)
    print(var_name + " values consecutive:" + str(len(var)-1 == 
                                df.loc[df[var_name].notna(), var_name].index[-1] - df.loc[df[var_name].notna(), var_name].index[0]))

m_altimeter_status values consecutive:False
m_digifin_status values consecutive:False
m_fin values consecutive:False
m_is_ballast_pump_moving values consecutive:False
m_is_battpos_moving values consecutive:False


#### Nav bay timebase

In [44]:
timebase,var=xbd.get("m_present_time")

In [66]:
pd.to_datetime(timebase, unit='s').sort_values()

DatetimeIndex([   '2023-11-09 07:07:55.036000',
               '2023-11-09 07:07:59.043000064',
               '2023-11-09 07:08:03.045000192',
               '2023-11-09 07:08:07.046999808',
               '2023-11-09 07:08:11.048999936',
               '2023-11-09 07:11:19.157999872',
               '2023-11-09 07:11:23.165999872',
               '2023-11-09 07:11:27.167000064',
               '2023-11-09 07:11:31.168999936',
                  '2023-11-09 07:11:35.172000',
               ...
               '2023-11-10 23:56:05.351000064',
               '2023-11-10 23:56:09.354000128',
               '2023-11-10 23:56:13.357000192',
               '2023-11-10 23:56:17.359000064',
               '2023-11-10 23:56:21.360999936',
                  '2023-11-10 23:56:25.364000',
               '2023-11-10 23:56:29.365999872',
               '2023-11-10 23:56:33.368999936',
                  '2023-11-10 23:56:37.372000',
               '2023-11-10 23:56:41.374000128'],
              dtype=

In [67]:
df = pd.DataFrame()
df["time"] = timebase
for var_name in noisy_variables:
    t,var=xbd.get(var_name)
    var_df = pd.DataFrame({"time":t, var_name:var})
    df = df.merge(var_df, how='outer', on="time")
df["time"] = pd.to_datetime(df["time"], unit='s')
df.sort_values(by=["time"], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0


In [68]:
for var_name in noisy_variables:
    print(var_name + " number of NaNs:" + str(df[var_name].isna().sum()))

m_altimeter_status number of NaNs:9812
m_digifin_status number of NaNs:0
m_fin number of NaNs:0
m_is_ballast_pump_moving number of NaNs:0
m_is_battpos_moving number of NaNs:0


In [69]:
for var_name in noisy_variables:
    t,var=xbd.get(var_name)
    print(var_name + " values consecutive:" + str(len(var)-1 == 
                                df.loc[df[var_name].notna(), var_name].index[-1] - df.loc[df[var_name].notna(), var_name].index[0]))

m_altimeter_status values consecutive:False
m_digifin_status values consecutive:True
m_fin values consecutive:True
m_is_ballast_pump_moving values consecutive:True
m_is_battpos_moving values consecutive:True


### Filling missing values

In [70]:
fill_cols = categoricals
df.loc[:,fill_cols] = df.loc[:,fill_cols].ffill().where(df.loc[:,fill_cols].bfill().notna()) # forward fill only in the middle

### Calculate change in appropriate variables

In [71]:
numeric_variables = [variable for variable in noisy_variables if variable not in categoricals] # ["m_ballast_pumped", "m_battpos", "m_fin"]

for var_name in numeric_variables:
    df[f"{var_name}_diff"] = df[var_name].diff()

df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000


#### Create corresponding categoricals

In [72]:
for var_name in numeric_variables:
    var_core = var_name.split('_')[1]
    df[f"is_{var_core}_moving"] = (df[f"{var_name}_diff"] != 0)
    df[f"is_{var_core}_moving"] = df[f"is_{var_core}_moving"].astype(float)
    df.loc[df[f"{var_name}_diff"].isna(), f"is_{var_core}_moving"] = pd.NA

# df["m_is_fin_moving"] = [float(diff != 0) for diff in df["m_fin_diff"]]
# df.loc[df["m_fin_diff"].isna(), "m_is_fin_moving"] = np.NaN
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0


## Distance to other glider

In [73]:
# Add coordinate data to dataframe
coordinate_variables = ["m_lat", "m_lon", "m_depth"]

for var_name in coordinate_variables:
    t,var=xbd.get(var_name)
    var_df = pd.DataFrame({"time":t, var_name:var})
    var_df["time"] = pd.to_datetime(var_df["time"], unit='s')
    df = df.merge(var_df, how='outer', on="time")
df.sort_values(by=["time"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [74]:
print(str(len(timebase)) + " total obs")
for var_name in coordinate_variables:
    t,var=xbd.get(var_name)
    print(var_name + " " + str(len(var)) + " obs")

32352 total obs


m_lat 28532 obs
m_lon 28532 obs
m_depth 29792 obs


In [75]:
# Interpolate missing values
df.set_index("time", inplace=True)

for var_name in coordinate_variables:
    df[var_name].interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")

df.reset_index(inplace=True)

In [76]:
UIVELO_DDIR = '../Example Glider Data/Binaries/Uivelo/'
UIVELO_FILE = ''
UIVELO_CDIR = '../Example Glider Data/Caches/Uivelo/'

file_format = '[de]bd'
uivelo_xbd = dbdreader.MultiDBD(pattern=UIVELO_DDIR+UIVELO_FILE+f'*.{file_format}', cacheDir=UIVELO_CDIR)
uivelo_xbd.set_time_limits("9 Nov 2023 00:00", "9 Nov 2023 23:00")

# Add Uivelo's coordinates to dataframe
for var_name in coordinate_variables:
    t,var=uivelo_xbd.get(var_name)
    var_df = pd.DataFrame({"time":t, f"uivelo_{var_name}":var})
    var_df["time"] = pd.to_datetime(var_df["time"], unit='s')
    df = df.merge(var_df, how='outer', on="time")
df.sort_values(by=["time"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [77]:
# Interpolate Uivelo's coordinates to Koskelo's timestamps
df.set_index("time", inplace=True)

for var_name in coordinate_variables:
    df[f"uivelo_{var_name}"].interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")

df.reset_index(inplace=True)

# Remove Uivelo's timestamps
df = df.loc[df["time"].isin(pd.to_datetime(timebase, unit='s'))]

df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,m_depth,uivelo_m_lat,uivelo_m_lon,uivelo_m_depth
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,0.000000,NaN,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,0.000000,NaN,NaN,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,0.000000,NaN,NaN,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,0.000000,NaN,NaN,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,0.000350,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35478,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,0.158856,NaN,NaN,NaN
35479,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,0.039250,NaN,NaN,NaN
35480,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN
35481,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
def haversine_distance(point1, point2):
    '''Haversine distance between two points, points in gps coordinates'''
    if(any(np.isnan(point1 + point2))):
        return np.NaN
    
    R = 6373.0
    lat1 = math.radians(point1[0])
    lon1 = math.radians(point1[1])
    lat2 = math.radians(point2[0])
    lon2 = math.radians(point2[1])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (math.sin(dlat/2))**2 + math.cos(lat1) * math.cos(lat2) * (math.sin(dlon/2))**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c
    return distance

In [79]:
df["glider_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row['uivelo_m_lat'],row['uivelo_m_lon']]), axis=1)
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,m_depth,uivelo_m_lat,uivelo_m_lon,uivelo_m_depth,glider_distance
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,0.000000,NaN,NaN,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,0.000000,NaN,NaN,NaN,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,0.000000,NaN,NaN,NaN,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,0.000000,NaN,NaN,NaN,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,0.000350,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35478,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,0.158856,NaN,NaN,NaN,NaN
35479,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,0.039250,NaN,NaN,NaN,NaN
35480,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35481,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Speed of sound

In [80]:
tctd, C, T, P = xbd.get_CTD_sync()

# Combine CTD and coordinates into one dataframe
sv_df = pd.DataFrame({"time":pd.to_datetime(tctd, unit='s'), "conductivity":C, "temperature":T, "pressure":P})
sv_df = sv_df.merge(df[["time", "m_lat", "m_lon"]], how="outer", on="time")
sv_df.sort_values(by=["time"], inplace=True)

# Set frame index to time
sv_df.set_index("time", inplace=True)

# Interpolate CTD based on time to fill nav timestamps with CTD values
for column in ["conductivity", "temperature", "pressure"]:
    sv_df[column].interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")

# Remove non-nav timestamps
sv_df = sv_df.loc[sv_df.index.isin(pd.to_datetime(timebase, unit='s'))]

# Remove rows with missing values
sv_df.dropna(inplace=True)

# Make time into a column again
sv_df.reset_index(inplace=True)

sv_df

,time,conductivity,temperature,pressure,m_lat,m_lon
0,2023-11-09 11:45:17.082000128,0.735910,8.570118,0.000370,59.819299,23.298168
1,2023-11-09 11:45:21.083000064,0.735912,8.570544,0.000372,59.819300,23.298166
2,2023-11-09 11:45:25.085999872,0.735900,8.569133,0.000433,59.819300,23.298163
3,2023-11-09 11:45:29.088000000,0.735900,8.569657,0.000000,59.819300,23.298161
4,2023-11-09 11:45:33.089999872,0.735880,8.568846,0.001614,59.819301,23.298159
...,...,...,...,...,...,...
29903,2023-11-10 23:54:04.519000064,0.734455,8.714209,-0.005910,59.741062,23.230303
29904,2023-11-10 23:54:08.520999936,0.734762,8.715391,-0.002056,59.741067,23.230307
29905,2023-11-10 23:54:12.524000000,0.734701,8.714814,-0.006808,59.741082,23.230312
29906,2023-11-10 23:54:16.528000000,0.734621,8.714517,-0.000042,59.741087,23.230320


In [81]:
C = sv_df["conductivity"]
T = sv_df["temperature"]
P = sv_df["pressure"]
lat = sv_df["m_lat"]
lon = sv_df["m_lon"]
SP = gsw.SP_from_C(C,T,P)
SA = gsw.SA_from_SP(SP,P,lon,lat)
CT = gsw.CT_from_t(SA,T,P)
rho = gsw.density.rho(SA,CT,P)
sv = gsw.sound_speed(SA,CT,P)
sv

0        1442.328211
1        1442.329946
2        1442.324188
3        1442.326318
4        1442.323020
            ...     
29903    1442.912545
29904    1442.917694
29905    1442.915218
29906    1442.914047
29907    1442.914311
Length: 29908, dtype: float64

In [82]:
sv_df["sound_speed"] = sv
df = df.merge(sv_df[["time","sound_speed"]], how="outer", on="time")
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,m_depth,uivelo_m_lat,uivelo_m_lon,uivelo_m_depth,glider_distance,sound_speed
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,0.000000,NaN,NaN,NaN,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,0.000000,NaN,NaN,NaN,NaN,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,0.000000,NaN,NaN,NaN,NaN,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,0.000000,NaN,NaN,NaN,NaN,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,0.000350,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,0.158856,NaN,NaN,NaN,NaN,NaN
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,0.039250,NaN,NaN,NaN,NaN,NaN
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
# Save
database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)

df["time"] = np.sort(timebase)
df.to_sql("gliders", db_connection, if_exists="replace", index=False)

db_connection.close()

In [95]:
# Load
database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)
query = ("SELECT * FROM gliders")

df = pd.read_sql_query(query, db_connection)
db_connection.close()

df["time"] = pd.to_datetime(df["time"], unit='s')
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,m_depth,uivelo_m_lat,uivelo_m_lon,uivelo_m_depth,glider_distance,sound_speed
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,0.000000,NaN,NaN,NaN,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,0.000000,NaN,NaN,NaN,NaN,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,0.000000,NaN,NaN,NaN,NaN,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,0.000000,NaN,NaN,NaN,NaN,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,0.000350,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,0.158856,NaN,NaN,NaN,NaN,NaN
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,0.039250,NaN,NaN,NaN,NaN,NaN
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Distance to ships

Note that current approach leaves out ship locations outside glider data time period, which can hinder their location interpolation.

Check below which tables would be better to use.

### Threats table

Note VIPs missing from threats table (Augusta, Aranda, although Augusta was added to VIPs mid-mission and is in threats at the start)

In [108]:
conn = sqlite3.connect("./Noise Data/AIS.sqlite")

# While the table has distance from glider values, we should calculate them again later due to merging and interpolation
query = ("SELECT mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught FROM threats "
            f"WHERE locAPICallTimestamp > '{min(df['time']).timestamp()*1000}' "
            f"AND locAPICallTimestamp < '{max(df['time']).timestamp()*1000}' "
            "AND distance_from_glider <= 10 "
            "AND glider_name = 'Koskelo'")                    

threats_df = pd.read_sql_query(query, conn)
threats_df["locUpdatetime"] = pd.to_datetime(threats_df["locUpdatetime"], format='ISO8601') # '%Y-%m-%dT%H:%M:%S.%f'

# Some rows don't have new location info (e.g. ship hasn't sent new data but glider has, so only distance value has changed)
threats_df.drop_duplicates(inplace=True)

conn.close()

threats_df

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught
0,230149210,AUGUSTA,23.274390,59.848840,2023-11-09 12:29:57.701,1699533001841,17.0
1,230149210,AUGUSTA,23.260058,59.812060,2023-11-09 12:40:01.342,1699533602711,17.0
2,230149210,AUGUSTA,23.178987,59.799055,2023-11-09 12:50:01.453,1699534202252,17.0
3,230149210,AUGUSTA,23.181612,59.821508,2023-11-09 13:00:00.900,1699534802693,17.0
4,230149210,AUGUSTA,23.232280,59.833880,2023-11-09 13:10:01.066,1699535402040,17.0
5,230149210,AUGUSTA,23.249552,59.845248,2023-11-09 13:19:59.452,1699536002723,17.0
6,230149210,AUGUSTA,23.249703,59.845043,2023-11-09 13:23:12.429,1699536602176,17.0
8,230992650,URSUS,23.140033,59.798222,2023-11-09 15:39:57.437,1699544402727,18.0
9,230992650,URSUS,23.178143,59.798652,2023-11-09 15:49:57.563,1699545002419,18.0
10,230992650,URSUS,23.214878,59.803878,2023-11-09 15:59:57.523,1699545601868,18.0


### Location + meta table

Everything within coordinate box

In [55]:
conn = sqlite3.connect("AIS2.sqlite") # AIS.sqlite

longitude_limits = [None, None]
latitude_limits = [None, None]

query = ("SELECT mmsi,name,longitude,latitude,locAPICallTimestamp,locUpdateTimestamp,draught FROM locations "
        "LEFT JOIN meta ON locations.mmsi = meta.mmsi "
        f"WHERE locations.mmsi IN "
            "(SELECT mmsi from locations "
            f"WHERE locAPICallTimestamp > '{min(df['time']).timestamp()*1000}' "
            f"AND locAPICallTimestamp < '{max(df['time']).timestamp()*1000}') "
            f"AND longitude > {longitude_limits[0]} "
            f"AND longitude < {longitude_limits[1]} "
            f"AND latitude > {latitude_limits[0]} "
            f"AND latitude < {latitude_limits[1]} "
        f"WHERE locAPICallTimestamp > '{min(df['time'])}' "
        f"AND locAPICallTimestamp < '{max(df['time'])}'")

ships_df = pd.read_sql_query(query, conn)
conn.close()

ships_df = ships_df.loc[:,~ships_df.columns.duplicated()] # If mmsi column duplicates from missing values in meta, drop the extra column

ships_df["locUpdatetime"] = pd.to_datetime(ships_df["locUpdateTimestamp"], unit="ms")

ships_df

DatabaseError: Execution failed on sql 'SELECT mmsi,longitude,latitude,locUpdatetime,draught FROM locations LEFT JOIN meta ON locations.mmsi = meta.mmsi WHERE locations.mmsi IN (SELECT mmsi from locations WHERE locAPICallTimestamp > '1699528662012.0' AND locAPICallTimestamp < '1699557409155.0') AND longitude > None AND longitude < None AND latitude > None AND latitude < None WHERE locAPICallTimestamp > '2023-11-09 11:17:42.012000' AND locAPICallTimestamp < '2023-11-09 19:16:49.155000064'': near "WHERE": syntax error

Just Augusta

In [86]:
ships_df = pd.DataFrame()

# Neither of these databases covers the whole mission and they overlap
# NOTE: only applies to full dataset, not this smaller example set
for db in ["AIS.sqlite"]: # , "AIS2.sqlite"
    conn = sqlite3.connect(f"./Noise Data/{db}")

    query = ("SELECT locations.mmsi AS mmsi,name,longitude,latitude,locAPICallTimestamp,locUpdateTimestamp,draught FROM locations "
            "LEFT JOIN meta ON locations.mmsi = meta.mmsi "
            f"WHERE locations.mmsi = 230149210 "
            f"AND locAPICallTimestamp > '{min(df['time']).timestamp()*1000}' "
            f"AND locAPICallTimestamp < '{max(df['time']).timestamp()*1000}' ")

    ships_df = pd.concat([ships_df, pd.read_sql_query(query, conn)], ignore_index=True)
    conn.close()

# ships_df = ships_df.loc[:,~ships_df.columns.duplicated()] # If mmsi column duplicates from missing values in meta, drop the extra column

ships_df.drop_duplicates(subset="locUpdateTimestamp",inplace=True) # Some API calls don't have new location info
                 
ships_df["locUpdatetime"] = pd.to_datetime(ships_df["locUpdateTimestamp"], unit="ms")

ships_df.sort_values(by=["locUpdatetime"], inplace=True)

ships_df

,mmsi,name,longitude,latitude,locAPICallTimestamp,locUpdateTimestamp,draught,locUpdatetime
0,230149210,AUGUSTA,23.274390,59.848840,1699533001841,1699532997701,17.0,2023-11-09 12:29:57.701
1,230149210,AUGUSTA,23.260058,59.812060,1699533602711,1699533601342,17.0,2023-11-09 12:40:01.342
2,230149210,AUGUSTA,23.178987,59.799055,1699534202252,1699534201453,17.0,2023-11-09 12:50:01.453
3,230149210,AUGUSTA,23.181612,59.821508,1699534802693,1699534800900,17.0,2023-11-09 13:00:00.900
4,230149210,AUGUSTA,23.232280,59.833880,1699535402040,1699535401066,17.0,2023-11-09 13:10:01.066
5,230149210,AUGUSTA,23.249552,59.845248,1699536002723,1699535999452,17.0,2023-11-09 13:19:59.452
6,230149210,AUGUSTA,23.249703,59.845043,1699536602176,1699536192429,17.0,2023-11-09 13:23:12.429


### Combine

With the smaller example data set, we can skip this step

In [ ]:
threats_df = threats_df.merge(ships_df, how="outer")

threats_df.drop(columns=['distance_from_glider','locUpdateTimestamp'], inplace=True)

threats_df.drop_duplicates(inplace=True)

threats_df.sort_values(by=["locAPICallTimestamp"], inplace=True)

threats_df

In [ ]:
threats_df.drop_duplicates(subset=["mmsi","locUpdatetime"],inplace=True)
threats_df

### Merge, fill and interpolate

#### Simple approach

##### EDA

Check if number of ships feasible first! If not, consider aggregation or the more advanced approach further below instead.

In [109]:
len(pd.unique(threats_df["mmsi"]))

10

In [110]:
print("Number of additional columns: " + str(len(threats_df.columns)*len(pd.unique(threats_df["mmsi"]))))
print("with " + str(len(df)) + " rows")

Number of additional columns: 70
with 32352 rows


With so many additional columns it seems better to use the other approach instead.

In [111]:
rows_per_mmsi = threats_df.groupby("mmsi").size()
rows_per_mmsi

mmsi
230149210    7
230622000    1
230631000    2
230642000    2
230992650    7
244130690    2
246554000    5
265004000    5
275517000    3
304677000    6
dtype: int64

In [112]:
rows_per_mmsi[rows_per_mmsi > 2]

mmsi
230149210    7
230992650    7
246554000    5
265004000    5
275517000    3
304677000    6
dtype: int64

##### Perform merge

In [93]:
for idx, mmsi in enumerate(pd.unique(threats_df["mmsi"])):
    # set ship data into timebase dataframe
    ship_df = threats_df.loc[threats_df["mmsi"] == mmsi].merge(
                pd.DataFrame({"time":pd.to_datetime(timebase, unit='s')}), 
                how='outer', left_on="locUpdatetime", right_on="time").copy()
    
    ship_df.loc[ship_df["time"].isna(), "time"] = ship_df.loc[ship_df["time"].isna(), "locUpdatetime"]
    
    # interpolate coordinates based on time
    ship_df.sort_values(by=["time"], inplace=True)
    ship_df.set_index("time", inplace=True)
    ship_df.latitude.interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")
    ship_df.longitude.interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")

    ship_df = ship_df.add_suffix(f"_{idx}")

    ship_df.reset_index(inplace=True)

    # fill categoricals
    fill_cols = [f"draught_{idx}", f"mmsi_{idx}"]
    ship_df.loc[:,fill_cols] = ship_df.loc[:,fill_cols].ffill().where(ship_df.loc[:,fill_cols].bfill().notna()) # forward fill only in the middle

    ship_df.dropna(subset=[f'longitude_{idx}', f'latitude_{idx}'])

    df = df.merge(ship_df, how='left', on="time")

df.sort_values(by=["time"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
df.loc[df["longitude_0"].notna(), "longitude_0"]

#### Dynamic approach

This approach, instead of making columns for each mmsi individually, assigns them to columns. The number of columns is based on the maximum number of ships active at the same time.

Example: If the data were to have at most 4 ships passing the glider at the same time, those 4 would be assigned to their own columns. All other ships would then also be assigned to those same columns based on where they fit.

##### EDA

In [114]:
# Which API calls had multiple ships?
duplicate_threats = threats_df[threats_df.duplicated(subset="locAPICallTimestamp", keep=False)]
duplicate_threats

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught
26,244130690,BORE SONG,23.094165,59.692772,2023-11-10 03:40:00.233,1699587602320,69.0
27,246554000,KRAFTCA,23.117130,59.710015,2023-11-10 03:39:57.280,1699587602320,73.0
28,244130690,BORE SONG,23.071433,59.724505,2023-11-10 03:49:59.784,1699588202192,69.0
29,246554000,KRAFTCA,23.050797,59.746190,2023-11-10 03:49:56.958,1699588202192,73.0


In [115]:
# What's the highest number of ships at once?
max_ships = max(duplicate_threats.groupby("locAPICallTimestamp").size())
max_ships

2

In [116]:
# Which ships had others with them most times?
ship_traffic_counts = duplicate_threats.groupby("mmsi").size().sort_values(ascending=False)
ship_traffic_counts

mmsi
244130690    2
246554000    2
dtype: int64

In [117]:
duplicate_threats.groupby("locAPICallTimestamp").size().sort_values()

locAPICallTimestamp
1699587602320    2
1699588202192    2
dtype: int64

In [118]:
max_ships_timestamps = duplicate_threats.groupby("locAPICallTimestamp").size()[duplicate_threats.groupby("locAPICallTimestamp").size() == 2].index
max_ships_timestamps

Index([1699587602320, 1699588202192], dtype='int64', name='locAPICallTimestamp')

In [119]:
max_traffic = duplicate_threats[duplicate_threats["locAPICallTimestamp"].isin(max_ships_timestamps)]
max_traffic

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught
26,244130690,BORE SONG,23.094165,59.692772,2023-11-10 03:40:00.233,1699587602320,69.0
27,246554000,KRAFTCA,23.117130,59.710015,2023-11-10 03:39:57.280,1699587602320,73.0
28,244130690,BORE SONG,23.071433,59.724505,2023-11-10 03:49:59.784,1699588202192,69.0
29,246554000,KRAFTCA,23.050797,59.746190,2023-11-10 03:49:56.958,1699588202192,73.0


In [120]:
# Which ships were in max ship situations?
max_ships_mmsis = max_traffic.mmsi.unique()
max_ships_mmsis

array([244130690, 246554000], dtype=int64)

In [121]:
# How many of these max ship situations was each one in?
max_traffic_counts = max_traffic.groupby("mmsi").size()
max_traffic_counts

mmsi
244130690    2
246554000    2
dtype: int64

In [122]:
# Which ships were in max traffic situations but not in most of them?
max_traffic[~max_traffic["mmsi"].isin(max_traffic_counts[max_traffic_counts == max(max_traffic_counts)].index)]

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught


In [123]:
# Which ships were in most max traffic situations and what were they doing?
threats_df[threats_df["mmsi"].isin(max_traffic_counts[max_traffic_counts == max(max_traffic_counts)].index)]

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught
24,246554000,KRAFTCA,23.315508,59.698627,2023-11-10 03:19:57.812,1699586402284,73.0
25,246554000,KRAFTCA,23.216355,59.702695,2023-11-10 03:29:57.154,1699587001955,73.0
26,244130690,BORE SONG,23.094165,59.692772,2023-11-10 03:40:00.233,1699587602320,69.0
27,246554000,KRAFTCA,23.117130,59.710015,2023-11-10 03:39:57.280,1699587602320,73.0
28,244130690,BORE SONG,23.071433,59.724505,2023-11-10 03:49:59.784,1699588202192,69.0
29,246554000,KRAFTCA,23.050797,59.746190,2023-11-10 03:49:56.958,1699588202192,73.0
36,246554000,KRAFTCA,23.069735,59.704688,2023-11-10 14:49:59.736,1699627802959,79.0


In [124]:
# How many times did each of them share space with another ship?
duplicate_threats[duplicate_threats["mmsi"].isin(max_ships_mmsis)].groupby("mmsi").size().sort_values(ascending=False)

mmsi
244130690    2
246554000    2
dtype: int64

In [125]:
duplicate_threats.groupby("locAPICallTimestamp").size()

locAPICallTimestamp
1699587602320    2
1699588202192    2
dtype: int64

##### Build dataframe

In [126]:
traffic_df = pd.DataFrame()
traffic_df["locAPICallTimestamp"] = threats_df["locAPICallTimestamp"].unique()
traffic_df.set_index("locAPICallTimestamp", inplace=True)
traffic_df

""
locAPICallTimestamp
1699533001841
1699533602711
1699534202252
1699534802693
1699535402040
1699536002723
1699536602176
1699544402727
1699545002419


In [127]:
added_ships = []

if(max_ships == len(max_ships_mmsis)):
    for i,mmsi in enumerate(max_ships_mmsis):
        ship_df = threats_df[threats_df["mmsi"] == mmsi]

        ship_df.set_index("locAPICallTimestamp", inplace=True)

        ship_df = ship_df.add_suffix(f"_{i}")
        traffic_df = traffic_df.merge(ship_df, how="left", on="locAPICallTimestamp")

        added_ships += [mmsi]

traffic_df.reset_index(inplace=True)
traffic_df

,locAPICallTimestamp,mmsi_0,name_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1
0,1699533001841,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,1699533602711,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,1699534202252,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,1699534802693,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,1699535402040,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
5,1699536002723,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,1699536602176,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,1699544402727,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,1699545002419,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
9,1699545601868,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [128]:
# Check unadded ships (that share space with another) # of times sharing space
ship_traffic_counts[~ship_traffic_counts.index.isin(max_ships_mmsis)]

Series([], dtype: int64)

In [129]:
unadded_ships = list(ship_traffic_counts[~ship_traffic_counts.index.isin(max_ships_mmsis)].index)
unadded_ships

[]

In [130]:
mmsi_cols = re.compile(".*mmsi.*")
mmsi_cols = list(filter(mmsi_cols.match, traffic_df.columns))
mmsi_cols

['mmsi_0', 'mmsi_1']

In [131]:
def add_to_df(traffic_df, added_ships, mmsi):
    for i in range(len(mmsi_cols)):
        # Check if current column fits this ship
        col_used_slots = traffic_df[traffic_df[f"mmsi_{i}"].notna()].index
        ship_potential_conflicts = duplicate_threats[duplicate_threats["mmsi"] == mmsi]["locAPICallTimestamp"]

        if(any(col_used_slots.isin(ship_potential_conflicts))):
            continue
        
        # If ship fits, add it
        ship_df = threats_df[threats_df["mmsi"] == mmsi]

        ship_df.set_index("locAPICallTimestamp", inplace=True)

        ship_df = ship_df.add_suffix(f"_{i}")
        traffic_df.update(ship_df, errors='raise')

        added_ships += [mmsi]
        return traffic_df, added_ships

traffic_df.set_index("locAPICallTimestamp", inplace=True)

for mmsi in unadded_ships:
    traffic_df, added_ships = add_to_df(traffic_df, added_ships, mmsi)

traffic_df.reset_index(inplace=True)
traffic_df

,locAPICallTimestamp,mmsi_0,name_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1
0,1699533001841,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,1699533602711,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,1699534202252,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,1699534802693,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,1699535402040,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
5,1699536002723,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,1699536602176,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,1699544402727,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,1699545002419,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
9,1699545601868,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [132]:
# Did we leave any potential conflicts unadded?
duplicate_threats[~duplicate_threats["mmsi"].isin(added_ships)]

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught


In [133]:
# Which ships had no conflicts and so aren't added yet?
threats_df[~threats_df["mmsi"].isin(ship_traffic_counts.index)]

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught
0,230149210,AUGUSTA,23.274390,59.848840,2023-11-09 12:29:57.701,1699533001841,17.0
1,230149210,AUGUSTA,23.260058,59.812060,2023-11-09 12:40:01.342,1699533602711,17.0
2,230149210,AUGUSTA,23.178987,59.799055,2023-11-09 12:50:01.453,1699534202252,17.0
3,230149210,AUGUSTA,23.181612,59.821508,2023-11-09 13:00:00.900,1699534802693,17.0
4,230149210,AUGUSTA,23.232280,59.833880,2023-11-09 13:10:01.066,1699535402040,17.0
5,230149210,AUGUSTA,23.249552,59.845248,2023-11-09 13:19:59.452,1699536002723,17.0
6,230149210,AUGUSTA,23.249703,59.845043,2023-11-09 13:23:12.429,1699536602176,17.0
8,230992650,URSUS,23.140033,59.798222,2023-11-09 15:39:57.437,1699544402727,18.0
9,230992650,URSUS,23.178143,59.798652,2023-11-09 15:49:57.563,1699545002419,18.0
10,230992650,URSUS,23.214878,59.803878,2023-11-09 15:59:57.523,1699545601868,18.0


In [134]:
no_conflict_mmsis = threats_df[~threats_df["mmsi"].isin(ship_traffic_counts.index)]["mmsi"].unique()
no_conflict_mmsis

array([230149210, 230992650, 304677000, 230622000, 230642000, 265004000,
       275517000, 230631000], dtype=int64)

In [135]:
traffic_df.set_index("locAPICallTimestamp", inplace=True)

for mmsi in no_conflict_mmsis:
    ship_df = threats_df[threats_df["mmsi"] == mmsi]

    ship_df.set_index("locAPICallTimestamp", inplace=True)

    ship_df = ship_df.add_suffix("_0")
    traffic_df.update(ship_df, errors='raise')

    added_ships += [mmsi]

traffic_df.reset_index(inplace=True)
traffic_df

,locAPICallTimestamp,mmsi_0,name_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1
0,1699533001841,230149210.0,AUGUSTA,23.274390,59.848840,2023-11-09 12:29:57.701,17.0,NaN,NaN,NaN,NaN,NaT,NaN
1,1699533602711,230149210.0,AUGUSTA,23.260058,59.812060,2023-11-09 12:40:01.342,17.0,NaN,NaN,NaN,NaN,NaT,NaN
2,1699534202252,230149210.0,AUGUSTA,23.178987,59.799055,2023-11-09 12:50:01.453,17.0,NaN,NaN,NaN,NaN,NaT,NaN
3,1699534802693,230149210.0,AUGUSTA,23.181612,59.821508,2023-11-09 13:00:00.900,17.0,NaN,NaN,NaN,NaN,NaT,NaN
4,1699535402040,230149210.0,AUGUSTA,23.232280,59.833880,2023-11-09 13:10:01.066,17.0,NaN,NaN,NaN,NaN,NaT,NaN
5,1699536002723,230149210.0,AUGUSTA,23.249552,59.845248,2023-11-09 13:19:59.452,17.0,NaN,NaN,NaN,NaN,NaT,NaN
6,1699536602176,230149210.0,AUGUSTA,23.249703,59.845043,2023-11-09 13:23:12.429,17.0,NaN,NaN,NaN,NaN,NaT,NaN
7,1699544402727,230992650.0,URSUS,23.140033,59.798222,2023-11-09 15:39:57.437,18.0,NaN,NaN,NaN,NaN,NaT,NaN
8,1699545002419,230992650.0,URSUS,23.178143,59.798652,2023-11-09 15:49:57.563,18.0,NaN,NaN,NaN,NaN,NaT,NaN
9,1699545601868,230992650.0,URSUS,23.214878,59.803878,2023-11-09 15:59:57.523,18.0,NaN,NaN,NaN,NaN,NaT,NaN


In [136]:
# Are there still any ships left unadded?
threats_df[~threats_df["mmsi"].isin(added_ships)]

,mmsi,name,longitude,latitude,locUpdatetime,locAPICallTimestamp,draught


Let's swap to using locAPICallTimestamp instead of individual locUpdatetimes for ease of use

In [171]:
traffic_df["locAPICallTime"] = pd.to_datetime(traffic_df["locAPICallTimestamp"], unit="ms")
traffic_df.drop(columns="locAPICallTimestamp", inplace=True)
traffic_df

,mmsi_0,name_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1,locAPICallTime
0,230149210.0,AUGUSTA,23.274390,59.848840,2023-11-09 12:29:57.701,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 12:30:01.841
1,230149210.0,AUGUSTA,23.260058,59.812060,2023-11-09 12:40:01.342,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 12:40:02.711
2,230149210.0,AUGUSTA,23.178987,59.799055,2023-11-09 12:50:01.453,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 12:50:02.252
3,230149210.0,AUGUSTA,23.181612,59.821508,2023-11-09 13:00:00.900,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:00:02.693
4,230149210.0,AUGUSTA,23.232280,59.833880,2023-11-09 13:10:01.066,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:10:02.040
5,230149210.0,AUGUSTA,23.249552,59.845248,2023-11-09 13:19:59.452,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:20:02.723
6,230149210.0,AUGUSTA,23.249703,59.845043,2023-11-09 13:23:12.429,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:30:02.176
7,230992650.0,URSUS,23.140033,59.798222,2023-11-09 15:39:57.437,18.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 15:40:02.727
8,230992650.0,URSUS,23.178143,59.798652,2023-11-09 15:49:57.563,18.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 15:50:02.419
9,230992650.0,URSUS,23.214878,59.803878,2023-11-09 15:59:57.523,18.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 16:00:01.868


Manually analyse difference between timestamps

In [138]:
max(traffic_df["locAPICallTime"] - traffic_df["locUpdatetime_0"])

Timedelta('0 days 00:06:49.747000')

In [139]:
traffic_df[traffic_df["locAPICallTime"] - traffic_df["locUpdatetime_0"] > timedelta(minutes=5)]

,locAPICallTimestamp,mmsi_0,name_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1,locAPICallTime
6,1699536602176,230149210.0,AUGUSTA,23.249703,59.845043,2023-11-09 13:23:12.429,17.0,NaN,NaN,NaN,NaN,NaT,NaN,2023-11-09 13:30:02.176


In [141]:
traffic_df[5:7]

,locAPICallTimestamp,mmsi_0,name_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1,locAPICallTime
5,1699536002723,230149210.0,AUGUSTA,23.249552,59.845248,2023-11-09 13:19:59.452,17.0,NaN,NaN,NaN,NaN,NaT,NaN,2023-11-09 13:20:02.723
6,1699536602176,230149210.0,AUGUSTA,23.249703,59.845043,2023-11-09 13:23:12.429,17.0,NaN,NaN,NaN,NaN,NaT,NaN,2023-11-09 13:30:02.176


Manually edit flawed values: remove if time since update is too long (unnecessary with example dataset)

In [44]:
# traffic_df.loc[210,["mmsi_0","name_0","longitude_0","latitude_0","locUpdatetime_0","draught_0"]] = pd.NA

In [172]:
# Save
# traffic_df.to_csv("traffic_df.csv", index=False)

# We'll want to convert the DatetimeIndex columns to string values for SQL storage
time_index_columns = re.compile("locUpdatetime.*")
time_index_columns = list(filter(time_index_columns.match, traffic_df.columns))
time_index_columns += ["locAPICallTime"]

traffic_df[time_index_columns] = traffic_df[time_index_columns].astype("str")

database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)

traffic_df.to_sql("traffic", db_connection, if_exists="replace", index=False)

db_connection.close()

In [173]:
# Load
""" traffic_df = pd.read_csv("traffic_df.csv", parse_dates=["locAPICallTime",
                                                        "locUpdatetime_0","locUpdatetime_1","locUpdatetime_2",
                                                        "locUpdatetime_3","locUpdatetime_4","locUpdatetime_5"]) """

database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)
query = ("SELECT * FROM traffic")

traffic_df = pd.read_sql_query(query, db_connection)
db_connection.close()

time_index_columns = re.compile("locUpdatetime.*")
time_index_columns = list(filter(time_index_columns.match, traffic_df.columns))
time_index_columns += ["locAPICallTime"]

for col in time_index_columns:
    traffic_df[col] = pd.to_datetime(traffic_df[col])
traffic_df

,mmsi_0,name_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1,locAPICallTime
0,230149210.0,AUGUSTA,23.274390,59.848840,2023-11-09 12:29:57.701,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 12:30:01.841
1,230149210.0,AUGUSTA,23.260058,59.812060,2023-11-09 12:40:01.342,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 12:40:02.711
2,230149210.0,AUGUSTA,23.178987,59.799055,2023-11-09 12:50:01.453,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 12:50:02.252
3,230149210.0,AUGUSTA,23.181612,59.821508,2023-11-09 13:00:00.900,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:00:02.693
4,230149210.0,AUGUSTA,23.232280,59.833880,2023-11-09 13:10:01.066,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:10:02.040
5,230149210.0,AUGUSTA,23.249552,59.845248,2023-11-09 13:19:59.452,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:20:02.723
6,230149210.0,AUGUSTA,23.249703,59.845043,2023-11-09 13:23:12.429,17.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 13:30:02.176
7,230992650.0,URSUS,23.140033,59.798222,2023-11-09 15:39:57.437,18.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 15:40:02.727
8,230992650.0,URSUS,23.178143,59.798652,2023-11-09 15:49:57.563,18.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 15:50:02.419
9,230992650.0,URSUS,23.214878,59.803878,2023-11-09 15:59:57.523,18.0,NaN,None,NaN,NaN,NaT,NaN,2023-11-09 16:00:01.868


##### Perform merge

In [174]:
df = df.merge(traffic_df, how="outer", left_on="time", right_on="locAPICallTime")
df.loc[df["time"].isna(), "time"] = df["locAPICallTime"]
df.drop(columns="locAPICallTime", inplace=True)
df.sort_values(by=["time"], inplace=True)
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [175]:
df.set_index("time", inplace=True)

mmsi_cols = re.compile(".*mmsi.*")
mmsi_cols = list(filter(mmsi_cols.match, traffic_df.columns))
mmsi_cols

for i in range(len(mmsi_cols)):
    # Forward fill categoricals and last location update time
    # Make sure not to fill rows between ships
    fill_mask = (df[f"mmsi_{i}"].ffill() == df[f"mmsi_{i}"].bfill())

    df[f"mmsi_{i}"] = df.loc[fill_mask, f"mmsi_{i}"].ffill()
    df[f"name_{i}"] = df.loc[fill_mask, f"name_{i}"].ffill()
    df[f"draught_{i}"] = df.loc[fill_mask, f"draught_{i}"].ffill()
    df[f"locUpdatetime_{i}"] = df.loc[fill_mask, f"locUpdatetime_{i}"].ffill()

    # Remove filled data if time since location update too long - avoid "interpolating forward" in the next step
    # Find location updates too long ago
    no_ffill_times = df.loc[df.index - df[f"locUpdatetime_{i}"] > timedelta(minutes=30), f"locUpdatetime_{i}"].unique()
    # Mark filled data after last location update for these occurences
    no_ffill_mask = pd.Series(df[f"locUpdatetime_{i}"].isin(no_ffill_times)).duplicated() & df[f"locUpdatetime_{i}"].isin(no_ffill_times)
    # Remove marked data
    df.loc[no_ffill_mask, [f"mmsi_{i}", f"name_{i}", f"draught_{i}"]] = pd.NA

    # Interpolate coordinates for each mmsi individually where we ffilled mmsi
    col_mmsis = df[f"mmsi_{i}"].unique()

    for mmsi in col_mmsis:
        # Create helper df for interpolation
        ship_df = df.loc[df[f"mmsi_{i}"] == mmsi, [f"latitude_{i}", f"longitude_{i}"]]
        # Interpolate coordinates
        ship_df.interpolate(method="time", limit_area="inside", inplace=True)
        # Update main df
        df.update(ship_df)

    # Remove categoricals where coordinates still NA
    df.loc[df[f"latitude_{i}"].isna(), [f"mmsi_{i}", f"name_{i}", f"draught_{i}"]] = pd.NA

df.reset_index(inplace=True)
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,locUpdatetime_1,draught_1
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32385,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
32386,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
32387,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN
32388,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN


Remove unnecessary columns

In [176]:
useless_cols = re.compile("locUpdatetime.*")
useless_cols = list(filter(useless_cols.match, traffic_df.columns))
useless_cols

['locUpdatetime_0', 'locUpdatetime_1']

In [177]:
df.drop(columns=useless_cols, inplace=True)
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,mmsi_0,name_0,longitude_0,latitude_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,draught_1
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32385,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32386,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32387,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32388,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
# Remove times not in timebase
df = df[df["time"].isin(pd.to_datetime(timebase, unit='s'))]
df.reset_index(drop=True, inplace=True)
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,mmsi_0,name_0,longitude_0,latitude_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,draught_1
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Calculate distances

Simple approach

In [ ]:
for idx, mmsi in enumerate(pd.unique(threats_df["mmsi"])):
    df[f"ship_{idx}_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row[f'latitude_{idx}'],row[f'longitude_{idx}']]), axis=1)
df

Dynamic approach

In [180]:
for idx in range(len(mmsi_cols)):
    df[f"ship_{idx}_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row[f'latitude_{idx}'],row[f'longitude_{idx}']]), axis=1)
df

C:\Users\vaisanee\AppData\Local\Temp\ipykernel_8576\3272324487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"ship_{idx}_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row[f'latitude_{idx}'],row[f'longitude_{idx}']]), axis=1)
C:\Users\vaisanee\AppData\Local\Temp\ipykernel_8576\3272324487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"ship_{idx}_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row[f'latitude_{idx}'],row

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,longitude_0,latitude_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,draught_1,ship_0_distance,ship_1_distance
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For consistency, remove Augusta's values where distance > 10km (unnecessary with example data set)

In [39]:
df.loc[df["ship_0_distance"] > 10,["mmsi_0","name_0","longitude_0","latitude_0","draught_0", "ship_0_distance"]] = pd.NA

## Deployment/retrieval vessel

In [181]:
boat_df = pd.DataFrame()

gpx_files = ["2023-11-09.gpx", "2023-11-09-2.gpx"]

for filename in gpx_files:
    boat_df = pd.concat([boat_df, pd.DataFrame(gpxtolist("./Noise Data/" + filename))], ignore_index=True)

boat_df = boat_df[["lat", "lon", "time"]]
boat_df["time"] = pd.to_datetime(boat_df["time"],format='%Y-%m-%dT%H:%M:%SZ')
boat_df

,lat,lon,time
0,59.816143,23.297432,2023-11-09 09:02:40
1,59.815983,23.297195,2023-11-09 09:05:44
2,59.816002,23.297088,2023-11-09 09:05:49
3,59.816037,23.296983,2023-11-09 09:05:55
4,59.816068,23.296888,2023-11-09 09:06:00
...,...,...,...
980,59.844938,23.249625,2023-11-09 14:11:06
981,59.844902,23.249528,2023-11-09 14:11:13
982,59.844912,23.249403,2023-11-09 14:13:07
983,59.844943,23.249303,2023-11-09 14:13:17


In [182]:
boat_df = boat_df.merge(pd.DataFrame({"time":pd.to_datetime(timebase, unit='s')}), how='outer', on="time")
    
boat_df.sort_values(by=["time"], inplace=True)
boat_df.reset_index(drop=True, inplace=True)
boat_df

,lat,lon,time
0,NaN,NaN,2023-11-09 07:07:55.036000000
1,NaN,NaN,2023-11-09 07:07:59.043000064
2,NaN,NaN,2023-11-09 07:08:03.045000192
3,NaN,NaN,2023-11-09 07:08:07.046999808
4,NaN,NaN,2023-11-09 07:08:11.048999936
...,...,...,...
33331,NaN,NaN,2023-11-10 23:56:25.364000000
33332,NaN,NaN,2023-11-10 23:56:29.365999872
33333,NaN,NaN,2023-11-10 23:56:33.368999936
33334,NaN,NaN,2023-11-10 23:56:37.372000000


Simple approach

In [ ]:
boat_df.set_index("time", inplace=True)
boat_df.lat.interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")
boat_df.lon.interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")
boat_df.reset_index(inplace=True)
boat_df = boat_df[boat_df["lat"].notna()]
boat_df

In [ ]:
df = df.merge(boat_df, how='left', on="time")
df

In [ ]:
df["boat_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row['lat'],row['lon']]), axis=1)
df

In [ ]:
df.loc[df["boat_distance"].notna(), "boat_distance"]

Dynamic approach

In [183]:
boat_df.set_index("time", inplace=True)
boat_df.lat.interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")
boat_df.lon.interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")
boat_df = boat_df[boat_df["lat"].notna()]
boat_df = boat_df[boat_df.index.isin(pd.to_datetime(timebase, unit='s'))] 
boat_df

,lat,lon
time,,
2023-11-09 09:02:43.418999808,59.816140,23.297428
2023-11-09 09:02:49.003000064,59.816135,23.297420
2023-11-09 09:02:53.005000192,59.816132,23.297415
2023-11-09 09:02:57.008999936,59.816128,23.297410
2023-11-09 09:03:01.012000000,59.816125,23.297405
...,...,...
2023-11-09 14:13:06.576999936,59.844912,23.249403
2023-11-09 14:13:10.579000064,59.844923,23.249367
2023-11-09 14:13:14.583000064,59.844936,23.249327


In [184]:
df.set_index("time", inplace=True)

for i in reversed(range(len(mmsi_cols))):
    if(all(df.loc[boat_df.index, f"latitude_{i}"].isna())):
        boat_df.rename(columns={"lat": f"latitude_{i}", "lon": f"longitude_{i}"}, inplace=True)
        boat_df[f"name_{i}"] = "Deployment/Retrieval"
        boat_df[f"mmsi_{i}"] = 1
        df.update(boat_df)
        print("Saved in column: " + str(i))

        df[f"ship_{i}_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row[f'latitude_{i}'],row[f'longitude_{i}']]), axis=1)
        
        break

df.reset_index(inplace=True)

Saved in column: 1


C:\Users\vaisanee\AppData\Local\Temp\ipykernel_8576\795367975.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"ship_{i}_distance"] = df.apply(lambda row: haversine_distance([row['m_lat'],row['m_lon']], [row[f'latitude_{i}'],row[f'longitude_{i}']]), axis=1)


In [185]:
df[df["time"].isin(boat_df.index)]

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,longitude_0,latitude_0,draught_0,mmsi_1,name_1,longitude_1,latitude_1,draught_1,ship_0_distance,ship_1_distance
67,2023-11-09 09:02:43.418999808,2.0,2.0,0.0,0.0,1.0,-0.002895,1.0,59.816123,23.297271,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.297428,59.816140,NaN,NaN,0.008968
68,2023-11-09 09:02:49.003000064,2.0,0.0,0.0,0.0,0.0,0.000000,0.0,59.816125,23.297280,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.297420,59.816135,NaN,NaN,0.007933
69,2023-11-09 09:02:53.005000192,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,59.816127,23.297277,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.297415,59.816132,NaN,NaN,0.007770
70,2023-11-09 09:02:57.008999936,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,59.816130,23.297273,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.297410,59.816128,NaN,NaN,0.007651
71,2023-11-09 09:03:01.012000000,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,59.816133,23.297272,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.297405,59.816125,NaN,NaN,0.007514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4011,2023-11-09 14:13:06.576999936,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,59.817777,23.299863,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.249403,59.844912,NaN,NaN,4.131025
4012,2023-11-09 14:13:10.579000064,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,59.817773,23.299859,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.249367,59.844923,NaN,NaN,4.133479
4013,2023-11-09 14:13:14.583000064,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,59.817765,23.299855,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.249327,59.844936,NaN,NaN,4.136515
4014,2023-11-09 14:13:18.584000000,1.0,0.0,0.0,1.0,0.0,0.000000,0.0,59.817760,23.299851,...,NaN,NaN,NaN,1.0,Deployment/Retrieval,23.249285,59.844949,NaN,NaN,4.139482


### Save data

In [187]:
# df.to_csv("traffic_noise_data.csv", index=False)

database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)

df["time"] = np.sort(timebase)
df.to_sql("noise", db_connection, if_exists="replace", index=False)

# At this point we can drop obsolete tables
def drop_table(db_connection, table):
    cursor = db_connection.cursor()
    cursor.execute(f'DROP TABLE {table}')
    db_connection.commit()

    db_connection.execute("VACUUM") # Reduce file size as well

for table in ["gliders", "traffic"]:
    drop_table(db_connection, table)

db_connection.close()

C:\Users\vaisanee\AppData\Local\Temp\ipykernel_8576\1544764811.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["time"] = np.sort(timebase)


In [188]:
# df = pd.read_csv("traffic_noise_data.csv")
# df["time"] = pd.to_datetime(df["time"])

database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)
query = ("SELECT * FROM noise")

df = pd.read_sql_query(query, db_connection)
db_connection.close()

df["time"] = pd.to_datetime(df["time"], unit='s')

## Weather data

### Find data

For this example, the weather data is already stored in the SQLite database since the FMI request doesn't work outside FMI.

In [50]:
""" #https://hav.fmi.fi/hav/asema/interface.php?interface=getcsvdata&fmisid=100932&lpnn=0101&dataset=fmisid_instant&begdate=09.11.2023&enddate=24.11.2023&quality_filter=1&separator=semicolon

fmisid = 100932
begdate = "09.11.2023"
enddate = "11.11.2023"
url = f"https://hav.fmi.fi/hav/asema/interface.php?interface=getcsvdata&fmisid={fmisid}&lpnn=0101&dataset=fmisid_instant&begdate={begdate}&enddate={enddate}&quality_filter=1&separator=semicolon"

response = requests.get(url)
    
if response.status_code == 200:
    data = response.text

    # Split the data into lines and then split each line by semicolon
    rows = data.split('\n')
    semicolon_separated_data = [row.split(';') for row in rows[:-2]]

    # Create a pandas DataFrame from the data
    weather_df = pd.DataFrame(semicolon_separated_data[1:], columns=semicolon_separated_data[0]) """

In [52]:
""" database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)

weather_df.to_sql("russarö_weather", db_connection, if_exists="replace", index=False)

db_connection.close() """

In [ ]:
database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)
query = ("SELECT * FROM russarö_weather")

weather_df = pd.read_sql_query(query, db_connection)

db_connection.close()

In [29]:
weather_df

,FMISID,LPNN,OBSDATE_UTC,OBSTIME_UTC,DATA_TIME,TA (degC),TD (degC),RH (%),PA (hPa),PA0,...,WG #2/1 25m (m/s),PRI avg 10M (mm/h),PRA max 10M (mm),WSD avg 10M #2 25m (m/s),WDD avg 10M #2 25m (deg),WGD max 10M #2 25m (m/s),CBST instant 10M,TCUST instant 10M,GSST instant 10M,GRST instant 10M
0,100932,0101,09.11.2023,00:00,2023-11-09 00:00:00,6.6,5.7,94,1003.6,1002.4,...,6.3,1.2,.09,5.5,196,6.3,2,1,2,0
1,100932,0101,09.11.2023,00:10,2023-11-09 00:10:00,6.8,6,94,1003.5,1002.4,...,6,.5,.06,5.3,197,6,2,1,2,0
2,100932,0101,09.11.2023,00:20,2023-11-09 00:20:00,6.9,6.1,95,1003.6,1002.5,...,5.6,.2,.03,4.4,187,5.6,1,2,1,0
3,100932,0101,09.11.2023,00:30,2023-11-09 00:30:00,7,6.1,94,1003.5,1002.4,...,5.7,0,0,4.6,185,5.7,2,1,2,0
4,100932,0101,09.11.2023,00:40,2023-11-09 00:40:00,7.1,6.1,94,1003.5,1002.3,...,7.1,0,0,5,184,7.1,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,100932,0101,11.11.2023,23:10,2023-11-11 23:10:00,6.9,6.6,98,1005.8,1004.6,...,6.7,.4,.07,5.8,53,6.7,0,2,0,0
428,100932,0101,11.11.2023,23:20,2023-11-11 23:20:00,6.8,6.6,99,1005.7,1004.6,...,6.8,.1,.02,6,54,6.8,0,0,0,0
429,100932,0101,11.11.2023,23:30,2023-11-11 23:30:00,6.8,6.6,98,1005.7,1004.5,...,6.5,0,0,5.8,53,6.5,0,0,0,0
430,100932,0101,11.11.2023,23:40,2023-11-11 23:40:00,6.8,6.6,98,1005.6,1004.5,...,6.3,.5,.06,5.6,54,6.3,0,0,0,0


In [30]:
russarö_df = weather_df[["DATA_TIME", "WS #2/1 25m (m/s)", "WD #2/1 25m (deg)", "WG #2/1 25m (m/s)", "WSP avg 10M #2/1 10m (m/s)"]]
russarö_df.columns = ["time", "russarö_wind_speed", "russarö_wind_direction", "russarö_gust_speed", "russarö_potential_wind"]
russarö_df

,time,russarö_wind_speed,russarö_wind_direction,russarö_gust_speed,russarö_potential_wind
0,2023-11-09 00:00:00,5.5,196,6.3,4.9
1,2023-11-09 00:10:00,5.3,197,6,4.8
2,2023-11-09 00:20:00,4.4,187,5.6,4
3,2023-11-09 00:30:00,4.6,185,5.7,4.1
4,2023-11-09 00:40:00,5,184,7.1,4.5
...,...,...,...,...,...
427,2023-11-11 23:10:00,5.8,53,6.7,5.2
428,2023-11-11 23:20:00,6,54,6.8,5.4
429,2023-11-11 23:30:00,5.8,53,6.5,5.2
430,2023-11-11 23:40:00,5.6,54,6.3,5


In [31]:
russarö_df["time"] = pd.to_datetime(russarö_df["time"],format='%Y-%m-%d %H:%M:%S')
russarö_df[["russarö_wind_speed", "russarö_wind_direction", 
            "russarö_gust_speed", "russarö_potential_wind"]] = russarö_df[["russarö_wind_speed", "russarö_wind_direction", 
                                                                           "russarö_gust_speed", "russarö_potential_wind"]].apply(pd.to_numeric)

C:\Users\vaisanee\AppData\Local\Temp\ipykernel_4620\2548967214.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  russarö_df["time"] = pd.to_datetime(russarö_df["time"],format='%Y-%m-%d %H:%M:%S')
C:\Users\vaisanee\AppData\Local\Temp\ipykernel_4620\2548967214.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  russarö_df[["russarö_wind_speed", "russarö_wind_direction",


In [32]:
russarö_df.dtypes

time                      datetime64[ns]
russarö_wind_speed               float64
russarö_wind_direction             int64
russarö_gust_speed               float64
russarö_potential_wind           float64
dtype: object

In [33]:
wind_speeds = re.compile(".*(m/s).*")
wind_speeds = list(filter(wind_speeds.match, weather_df.columns))
wind_speeds

['WS #2/1 25m (m/s)',
 'WSP avg 10M #2/1 10m (m/s)',
 'WG #2/1 25m (m/s)',
 'WSD avg 10M #2 25m (m/s)',
 'WGD max 10M #2 25m (m/s)']

In [53]:
""" url = "https://hav.fmi.fi/hav/asema/interface.php?interface=getcsvdata&fmisid=100965&lpnn=0215&dataset=fmisid_instant_1min&begdate=09.11.2023&enddate=11.11.2023&quality_filter=1&separator=semicolon"

response = requests.get(url)
    
if response.status_code == 200:
    data = response.text

    # Split the data into lines and then split each line by semicolon
    rows = data.split('\n')
    semicolon_separated_data = [row.split(';') for row in rows[:-2]]

    # Create a pandas DataFrame from the data
    weather_df = pd.DataFrame(semicolon_separated_data[1:], columns=semicolon_separated_data[0]) """

In [54]:
""" database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)

weather_df.to_sql("jussarö_weather", db_connection, if_exists="replace", index=False)

db_connection.close() """

In [ ]:
database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)
query = ("SELECT * FROM jussarö_weather")

weather_df = pd.read_sql_query(query, db_connection)

db_connection.close()

In [35]:
jussarö_df = weather_df[["DATA_TIME", "WS #3/1 21.3m (m/s)", "WD #3/1 21.3m (deg)", "WG #3/1 21.3m (m/s)", "WSP avg 10M #3/1 10m (m/s)"]]
jussarö_df.columns = ["time", "jussarö_wind_speed", "jussarö_wind_direction", "jussarö_gust_speed", "jussarö_potential_wind"]
jussarö_df

,time,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind
0,2023-11-09 00:00:00,5.2,148,7.6,4.7
1,2023-11-09 00:01:00,4.6,150,7.6,
2,2023-11-09 00:02:00,4,153,7.4,
3,2023-11-09 00:03:00,3.6,160,7.4,
4,2023-11-09 00:04:00,3.4,177,6.6,
...,...,...,...,...,...
4315,2023-11-11 23:55:00,7.2,81,8.3,
4316,2023-11-11 23:56:00,7.2,81,8.3,
4317,2023-11-11 23:57:00,7.2,81,8.3,
4318,2023-11-11 23:58:00,7.1,82,8.3,


In [36]:
jussarö_df["time"] = pd.to_datetime(jussarö_df["time"],format='%Y-%m-%d %H:%M:%S')
jussarö_df[["jussarö_wind_speed", "jussarö_wind_direction", 
            "jussarö_gust_speed", "jussarö_potential_wind"]] = jussarö_df[["jussarö_wind_speed", "jussarö_wind_direction", 
                                                                           "jussarö_gust_speed", "jussarö_potential_wind"]].apply(pd.to_numeric)

C:\Users\vaisanee\AppData\Local\Temp\ipykernel_4620\316880716.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jussarö_df["time"] = pd.to_datetime(jussarö_df["time"],format='%Y-%m-%d %H:%M:%S')
C:\Users\vaisanee\AppData\Local\Temp\ipykernel_4620\316880716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jussarö_df[["jussarö_wind_speed", "jussarö_wind_direction",


In [37]:
jussarö_df.dtypes

time                      datetime64[ns]
jussarö_wind_speed               float64
jussarö_wind_direction             int64
jussarö_gust_speed               float64
jussarö_potential_wind           float64
dtype: object

In [38]:
jussarö_df[jussarö_df["jussarö_potential_wind"].notna()]

,time,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind
0,2023-11-09 00:00:00,5.2,148,7.6,4.7
10,2023-11-09 00:10:00,3.4,259,5.1,3.1
20,2023-11-09 00:20:00,2.6,254,4.2,2.3
30,2023-11-09 00:30:00,2.2,249,4.1,2.0
40,2023-11-09 00:40:00,3.6,248,5.1,3.2
...,...,...,...,...,...
4270,2023-11-11 23:10:00,6.1,78,7.1,5.5
4280,2023-11-11 23:20:00,6.5,76,8.0,5.8
4290,2023-11-11 23:30:00,7.0,77,8.3,6.3
4300,2023-11-11 23:40:00,6.9,78,8.0,6.2


### Merge data

Let's just use the 10min timestep for simplicity

In [39]:
weather_df = jussarö_df.merge(russarö_df, how='inner', on="time")
weather_df

,time,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind,russarö_wind_speed,russarö_wind_direction,russarö_gust_speed,russarö_potential_wind
0,2023-11-09 00:00:00,5.2,148,7.6,4.7,5.5,196,6.3,4.9
1,2023-11-09 00:10:00,3.4,259,5.1,3.1,5.3,197,6.0,4.8
2,2023-11-09 00:20:00,2.6,254,4.2,2.3,4.4,187,5.6,4.0
3,2023-11-09 00:30:00,2.2,249,4.1,2.0,4.6,185,5.7,4.1
4,2023-11-09 00:40:00,3.6,248,5.1,3.2,5.0,184,7.1,4.5
...,...,...,...,...,...,...,...,...,...
427,2023-11-11 23:10:00,6.1,78,7.1,5.5,5.8,53,6.7,5.2
428,2023-11-11 23:20:00,6.5,76,8.0,5.8,6.0,54,6.8,5.4
429,2023-11-11 23:30:00,7.0,77,8.3,6.3,5.8,53,6.5,5.2
430,2023-11-11 23:40:00,6.9,78,8.0,6.2,5.6,54,6.3,5.0


Move to glider timebase and interpolate

In [45]:
weather_df = weather_df.merge(pd.DataFrame({"time":pd.to_datetime(timebase, unit='s')}), how='outer', on="time")
    
weather_df.sort_values(by=["time"], inplace=True)

weather_df.set_index("time", inplace=True)

for column in weather_df.columns:
    weather_df[column].interpolate(method='time', inplace=True, limit_direction="forward", limit_area="inside")

weather_df = weather_df[weather_df.index.isin(pd.to_datetime(timebase, unit='s'))] 
weather_df.reset_index(inplace=True)
weather_df

,time,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind,russarö_wind_speed,russarö_wind_direction,russarö_gust_speed,russarö_potential_wind
0,2023-11-09 07:07:55.036000000,3.712554,246.791727,5.637518,3.333381,5.445791,262.000000,6.649928,4.924964
1,2023-11-09 07:07:59.043000064,3.718565,246.798405,5.639522,3.338724,5.441116,262.000000,6.641914,4.920957
2,2023-11-09 07:08:03.045000192,3.724568,246.805075,5.641523,3.344060,5.436447,262.000000,6.633910,4.916955
3,2023-11-09 07:08:07.046999808,3.730570,246.811745,5.643523,3.349396,5.431779,262.000000,6.625906,4.912953
4,2023-11-09 07:08:11.048999936,3.736573,246.818415,5.645524,3.354732,5.427110,262.000000,6.617902,4.908951
...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,6.778863,185.642273,8.478863,5.543091,7.900000,185.926820,9.050409,7.000000
32348,2023-11-10 23:56:29.365999872,6.775528,185.648943,8.475528,5.540423,7.900000,185.946830,9.045740,7.000000
32349,2023-11-10 23:56:33.368999936,6.772193,185.655615,8.472193,5.537754,7.900000,185.966845,9.041070,7.000000
32350,2023-11-10 23:56:37.372000000,6.768857,185.662287,8.468857,5.535085,7.900000,185.986860,9.036399,7.000000


In [48]:
df = df.merge(weather_df, on="time")
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,ship_0_distance,ship_1_distance,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind,russarö_wind_speed,russarö_wind_direction,russarö_gust_speed,russarö_potential_wind
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,NaN,3.712554,246.791727,5.637518,3.333381,5.445791,262.000000,6.649928,4.924964
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,...,NaN,NaN,3.718565,246.798405,5.639522,3.338724,5.441116,262.000000,6.641914,4.920957
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,...,NaN,NaN,3.724568,246.805075,5.641523,3.344060,5.436447,262.000000,6.633910,4.916955
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,...,NaN,NaN,3.730570,246.811745,5.643523,3.349396,5.431779,262.000000,6.625906,4.912953
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,...,NaN,NaN,3.736573,246.818415,5.645524,3.354732,5.427110,262.000000,6.617902,4.908951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,...,NaN,NaN,6.778863,185.642273,8.478863,5.543091,7.900000,185.926820,9.050409,7.000000
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,...,NaN,NaN,6.775528,185.648943,8.475528,5.540423,7.900000,185.946830,9.045740,7.000000
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,6.772193,185.655615,8.472193,5.537754,7.900000,185.966845,9.041070,7.000000
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,6.768857,185.662287,8.468857,5.535085,7.900000,185.986860,9.036399,7.000000


## Save data

In [56]:
# df.to_csv("TVAR20233_noise_data.csv", index=False)

database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)

df["time"] = np.sort(timebase)
df.to_sql("noise", db_connection, if_exists="replace", index=False)

db_connection.close()

## Visualisation

In [57]:
# df = pd.read_csv("TVAR20233_noise_data.csv")

database = "./Noise Data/TVAR20233_noise_data.sqlite"
db_connection = sqlite3.connect(database)
query = ("SELECT * FROM noise")

df = pd.read_sql_query(query, db_connection)
db_connection.close()

df["time"] = pd.to_datetime(df["time"], unit='s')
df

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,ship_0_distance,ship_1_distance,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind,russarö_wind_speed,russarö_wind_direction,russarö_gust_speed,russarö_potential_wind
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,NaN,3.712554,246.791727,5.637518,3.333381,5.445791,262.000000,6.649928,4.924964
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249485,...,NaN,NaN,3.718565,246.798405,5.639522,3.338724,5.441116,262.000000,6.641914,4.920957
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,59.844882,23.249483,...,NaN,NaN,3.724568,246.805075,5.641523,3.344060,5.436447,262.000000,6.633910,4.916955
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844882,23.249482,...,NaN,NaN,3.730570,246.811745,5.643523,3.349396,5.431779,262.000000,6.625906,4.912953
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.000000,0.0,59.844880,23.249475,...,NaN,NaN,3.736573,246.818415,5.645524,3.354732,5.427110,262.000000,6.617902,4.908951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32347,2023-11-10 23:56:25.364000000,2.0,2.0,0.112899,0.0,0.0,0.115794,1.0,NaN,NaN,...,NaN,NaN,6.778863,185.642273,8.478863,5.543091,7.900000,185.926820,9.050409,7.000000
32348,2023-11-10 23:56:29.365999872,2.0,2.0,-0.002895,0.0,0.0,-0.115794,1.0,NaN,NaN,...,NaN,NaN,6.775528,185.648943,8.475528,5.540423,7.900000,185.946830,9.045740,7.000000
32349,2023-11-10 23:56:33.368999936,2.0,0.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,6.772193,185.655615,8.472193,5.537754,7.900000,185.966845,9.041070,7.000000
32350,2023-11-10 23:56:37.372000000,2.0,2.0,-0.002895,0.0,0.0,0.000000,0.0,NaN,NaN,...,NaN,NaN,6.768857,185.662287,8.468857,5.535085,7.900000,185.986860,9.036399,7.000000


In [58]:
df["surface_depth"] = 0

The first row clearly has flawed coordinate data, remove it

In [59]:
df[df["m_lat"].notna()]

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,ship_1_distance,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind,russarö_wind_speed,russarö_wind_direction,russarö_gust_speed,russarö_potential_wind,surface_depth
0,2023-11-09 07:07:55.036000000,1.0,0.0,0.000000,1.0,1.0,NaN,NaN,60.203570,24.960154,...,NaN,3.712554,246.791727,5.637518,3.333381,5.445791,262.000000,6.649928,4.924964,0
1,2023-11-09 07:07:59.043000064,1.0,0.0,0.000000,1.0,0.0,0.0,0.0,59.844882,23.249485,...,NaN,3.718565,246.798405,5.639522,3.338724,5.441116,262.000000,6.641914,4.920957,0
2,2023-11-09 07:08:03.045000192,1.0,0.0,0.000000,1.0,0.0,0.0,0.0,59.844882,23.249483,...,NaN,3.724568,246.805075,5.641523,3.344060,5.436447,262.000000,6.633910,4.916955,0
3,2023-11-09 07:08:07.046999808,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,59.844882,23.249482,...,NaN,3.730570,246.811745,5.643523,3.349396,5.431779,262.000000,6.625906,4.912953,0
4,2023-11-09 07:08:11.048999936,2.0,0.0,0.000000,0.0,0.0,0.0,0.0,59.844880,23.249475,...,NaN,3.736573,246.818415,5.645524,3.354732,5.427110,262.000000,6.617902,4.908951,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32312,2023-11-10 23:54:04.519000064,2.0,0.0,-0.002895,0.0,0.0,0.0,0.0,59.741062,23.230303,...,NaN,6.896234,185.407532,8.596234,5.636987,7.900000,185.222595,9.214728,7.000000,0
32313,2023-11-10 23:54:08.520999936,2.0,0.0,-0.002895,0.0,0.0,0.0,0.0,59.741067,23.230307,...,NaN,6.892899,185.414202,8.592899,5.634319,7.900000,185.242605,9.210059,7.000000,0
32314,2023-11-10 23:54:12.524000000,2.0,0.0,-0.002895,0.0,0.0,0.0,0.0,59.741082,23.230312,...,NaN,6.889563,185.420873,8.589563,5.631651,7.900000,185.262620,9.205389,7.000000,0
32315,2023-11-10 23:54:16.528000000,2.0,0.0,-0.002895,0.0,0.0,0.0,0.0,59.741087,23.230320,...,NaN,6.886227,185.427547,8.586227,5.628981,7.900000,185.282640,9.200717,7.000000,0


In [60]:
df.loc[0,["m_lat","m_lon"]] = pd.NA
df.loc[0]

time                        2023-11-09 07:07:55.036000
m_altimeter_status                                 1.0
m_digifin_status                                   0.0
m_fin                                              0.0
m_is_ballast_pump_moving                           1.0
m_is_battpos_moving                                1.0
m_fin_diff                                         NaN
is_fin_moving                                      NaN
m_lat                                              NaN
m_lon                                              NaN
m_depth                                            0.0
uivelo_m_lat                                       NaN
uivelo_m_lon                                       NaN
uivelo_m_depth                                     NaN
glider_distance                                    NaN
sound_speed                                        NaN
mmsi_0                                             NaN
name_0                                            None
longitude_

In [61]:
latitudes = re.compile(".*lat.*")
latitudes = list(filter(latitudes.match, df.columns))

longitudes = re.compile(".*lon.*")
longitudes = list(filter(longitudes.match, df.columns))

max_lat = np.nanmax(df[latitudes])
min_lat = np.nanmin(df[latitudes])

max_lon = np.nanmax(df[longitudes])
min_lon = np.nanmin(df[longitudes])

max_depth = np.nanmax(df[["m_depth", "uivelo_m_depth"]])

In [62]:
mean_lat_radians = np.radians((max_lat + min_lat)/2)
lon_lat_rel = np.cos(mean_lat_radians)

depth_lat_rel = 1/(1000*6371*np.pi/180)

In [63]:
print("Lat:Lon:Depth = 1 : " + str(lon_lat_rel) + " : " + str(depth_lat_rel))

Lat:Lon:Depth = 1 : 0.5035603368068401 : 8.993216059187306e-06


In [64]:
mmsi_cols = re.compile(".*mmsi.*")
mmsi_cols = list(filter(mmsi_cols.match, df.columns))
mmsi_cols

['mmsi_0', 'mmsi_1']

In [65]:
wind_speeds = re.compile(".*wind_speed")
wind_speeds = list(filter(wind_speeds.match, df.columns))

max_wind_speed = max(df[wind_speeds].max())

In [66]:
noise_df = df.copy()

In [75]:
df = noise_df[(noise_df["time"] > datetime.strptime("2023-11-09 08:00","%Y-%m-%d %H:%M")) & 
              (noise_df["time"] < datetime.strptime("2023-11-10 00:00","%Y-%m-%d %H:%M"))]

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="3d-scatter-plot-x-graph"),
    html.P("Time"),
    dcc.RangeSlider(
        id='3d-scatter-plot-t-range-slider',
        # updatemode='drag',
        min=min(df.index), max=max(df.index), step=1,
        marks=None, # df['time'][idx] # {idx:'' for idx in range(len(df))}
        tooltip={"placement": "bottom", "always_visible": True},
        value=[min(df.index)]
    ),
    html.P("Longitude"),
    dcc.RangeSlider(
        id='3d-scatter-plot-x-range-slider',
        min=round(min_lon-0.01,2), max=round(max_lon+0.01,2), step=0.01,
        value=[round(min_lon-0.01,2), round(max_lon+0.01,2)],
        marks=None,
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    html.P("Latitude"),
    dcc.RangeSlider(
        id='3d-scatter-plot-y-range-slider',
        min=round(min_lat-0.01,2), max=round(max_lat+0.01,2), step=0.005,
        value=[round(min_lat-0.01,2), round(max_lat+0.01,2)],
        marks=None,
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    html.P("Depth"),
    dcc.RangeSlider(
        id='3d-scatter-plot-z-range-slider',
        min=0, max=np.ceil(max_depth), step=1,
        value=[0, np.ceil(max_depth)],
        marks=None,
        tooltip={"placement": "bottom", "always_visible": True}
    ),
    html.P('Camera controls'),
    html.Div([
        dcc.Input(id='input-eye-x-state', type='number', placeholder="Eye X"),
        dcc.Input(id='input-eye-y-state', type='number', placeholder="Eye Y"),
        dcc.Input(id='input-eye-z-state', type='number', placeholder="Eye Z"),
        html.Button(id='submit-button-eye-state', n_clicks=0, children='Submit')
    ]),
    html.Div([
        dcc.Input(id='input-center-x-state', type='number', placeholder="Center X"),
        dcc.Input(id='input-center-y-state', type='number', placeholder="Center Y"),
        dcc.Input(id='input-center-z-state', type='number', placeholder="Center Z"),
        html.Button(id='submit-button-center-state', n_clicks=0, children='Submit')
    ]),
    html.Div([dag.AgGrid(
            id="value-table",
            columnDefs=[{"field": f"col{i}", "width": (i*100 % 200)+110} for i in range(1,9)],
            #columnSize="responsiveSizeToFit",
            dashGridOptions={"headerHeight":0},
            style={"height": 220, "width": "100%"}
       )]),
])

@app.callback(
    Output("value-table", "rowData"),
    Input("3d-scatter-plot-t-range-slider", "value"))


def update_value_tables(slider_t):
    rowData = [{'col1': "m_altimeter_status",     'col2': df.loc[slider_t, "m_altimeter_status"], 
                'col3': "russarö_wind_direction", 'col4': round(df.loc[slider_t, "russarö_wind_direction"], 1),
                'col5': "jussarö_wind_direction", 'col6': round(df.loc[slider_t, "jussarö_wind_direction"], 1),
                'col7': "sound_speed",            'col8': round(df.loc[slider_t, "sound_speed"], 1)}, 
               {'col1': "m_digifin_status",   'col2': df.loc[slider_t, "m_digifin_status"], 
                'col3': "russarö_wind_speed", 'col4': round(df.loc[slider_t, "russarö_wind_speed"], 1),
                'col5': "jussarö_wind_speed", 'col6': round(df.loc[slider_t, "jussarö_wind_speed"], 1)},
               {'col1': "is_fin_moving",      'col2': df.loc[slider_t, "is_fin_moving"], 
                'col3': "russarö_gust_speed", 'col4': round(df.loc[slider_t, "russarö_gust_speed"], 1),
                'col5': "jussarö_gust_speed", 'col6': round(df.loc[slider_t, "jussarö_gust_speed"], 1)},
               {'col1': "m_is_ballast_pump_moving", 'col2': df.loc[slider_t, "m_is_ballast_pump_moving"], 
                'col3': "russarö_potential_wind",   'col4': round(df.loc[slider_t, "russarö_potential_wind"], 1),
                'col5': "jussarö_potential_wind",   'col6': round(df.loc[slider_t, "jussarö_potential_wind"], 1)},
               {'col1': "m_is_battpos_moving", 'col2': df.loc[slider_t, "m_is_battpos_moving"]}]
    
    return rowData


@app.callback(
    Output("3d-scatter-plot-x-graph", "figure"),
    [Input("3d-scatter-plot-t-range-slider", "value"),
    Input("3d-scatter-plot-x-range-slider", "value"),
    Input("3d-scatter-plot-y-range-slider", "value"),
    Input("3d-scatter-plot-z-range-slider", "value"),
    Input('submit-button-eye-state', 'n_clicks'),
    State('input-eye-x-state', 'value'),
    State('input-eye-y-state', 'value'),
    State('input-eye-z-state', 'value'),
    Input('submit-button-center-state', 'n_clicks'),
    State('input-center-x-state', 'value'),
    State('input-center-y-state', 'value'),
    State('input-center-z-state', 'value')])

def update_trajectory_chart(slider_t, slider_x, slider_y, slider_z, 
                            n_eye_clicks, eye_x, eye_y, eye_z, 
                            n_center_clicks, center_x, center_y, center_z):
    location_t = slider_t
    low_x, high_x = slider_x
    low_y, high_y = slider_y
    low_z, high_z = slider_z
    
    fig = go.Figure()
    
    # Define glider trace styles
    trace_style_list = [{"name":"Koskelo", 
                         "marker_symbol":"cross", "marker_color":"blue", "marker_size":20,
                         "line_color":"dodgerblue",
                         "x":"m_lon", "y":"m_lat", "z":"m_depth", 
                         "text":None,
                         "mask":[True]*len(df)},
                        {"name":"Uivelo", 
                         "marker_symbol":"cross", "marker_color":"red", "marker_size":20,
                         "line_color":"lightcoral",
                         "x":"uivelo_m_lon", "y":"uivelo_m_lat", "z":"uivelo_m_depth",
                         "text":"Distance: " + str(round(df.loc[location_t, "glider_distance"].values[0],2)) + " km",
                         "mask":df["uivelo_m_lat"].notna()}]
    
    # Define ship trace styles
    # Colors
    vessel_trace_colors_list = [{"marker_color":"seagreen","line_color":"limegreen"},
                                {"marker_color":"purple","line_color":"violet"},
                                {"marker_color":"orange","line_color":"goldenrod"},
                                {"marker_color":"springgreen","line_color":"turquoise"},
                                {"marker_color":"brown","line_color":"chocolate"},
                                {"marker_color":"peru","line_color":"maroon"}]
    # Column dependent values
    for i,mmsi_col in enumerate(mmsi_cols):
        mmsi = df.loc[location_t, mmsi_col].values[0]
        
        # If this ship column is empty, skip it
        if(np.isnan(mmsi)): 
            continue

        trace_style_list += [{"name":df.loc[location_t, f"name_{i}"].values[0], 
                              "marker_symbol":"diamond", "marker_size":8,
                              "marker_color":vessel_trace_colors_list[i]["marker_color"],
                              "line_color":vessel_trace_colors_list[i]["line_color"],
                              "x":f"longitude_{i}", "y":f"latitude_{i}", "z":"surface_depth",
                              "text":"Distance: " + str(round(df.loc[location_t, f"ship_{i}_distance"].values[0],2)) + " km",
                              "mask":(df[f"mmsi_{i}"] == mmsi)}]
    
    # Create linepaths
    for trace in trace_style_list:
        fig.add_trace(go.Scatter3d(x=df.loc[trace['mask'], trace['x']], 
                                   y=df.loc[trace['mask'], trace['y']], 
                                   z=df.loc[trace['mask'], trace['z']], 
                            mode='lines', name=trace['name'], line_color=trace['line_color']))

    # Create markers
    for trace in trace_style_list:
        fig.add_trace(
            go.Scatter3d(
                mode="markers",
                marker_color=trace['marker_color'],
                marker_size=trace['marker_size'],
                marker_symbol=trace['marker_symbol'],
                name=trace['name'] + " at this time",
                x=df.loc[location_t, trace['x']],
                y=df.loc[location_t, trace['y']],
                z=df.loc[location_t, trace['z']],
                text=trace['text']))
    
    # Create wind cones
    r_wind_dir = np.radians(df.loc[location_t, "russarö_wind_direction"].values[0])
    r_wind_spd = df.loc[location_t, "russarö_wind_speed"].values[0]
    j_wind_dir = np.radians(df.loc[location_t, "jussarö_wind_direction"].values[0])
    j_wind_spd = df.loc[location_t, "jussarö_wind_speed"].values[0]
    
    # NOTE: The cones need scaling to both fit the plot and match the axis aspect ratio
    fig.add_trace(go.Cone(x=[low_x+0.01], y=[(low_y+high_y)/2], z=[(low_z+high_z)/2], 
                          u=[r_wind_spd*np.sin(r_wind_dir)], v=[r_wind_spd*np.cos(r_wind_dir)*lon_lat_rel], w=[0],
                          text="Russarö wind speed: " + str(round(r_wind_spd,1)) + " m/s", hoverinfo="text",
                          showscale=False, cmin=0, cmax=max_wind_speed, colorscale="Electric",
                          sizemode="scaled", sizeref=1/100)
                 )
    fig.add_trace(go.Cone(x=[high_x-0.01], y=[(low_y+high_y)/2], z=[(low_z+high_z)/2], 
                          u=[j_wind_spd*np.sin(j_wind_dir)], v=[j_wind_spd*np.cos(j_wind_dir)*lon_lat_rel], w=[0],
                          text="Jussarö wind speed: " + str(round(j_wind_spd,1)) + " m/s", hoverinfo="text",
                          showscale=False, cmin=0, cmax=max_wind_speed, colorscale="Electric",
                          sizemode="scaled", sizeref=1/100)
                 )

    fig.update_layout(title_text=str(df.loc[location_t[0], "time"]),
                      scene=dict(xaxis = dict(range=[low_x, high_x]),
                                yaxis = dict(range=[low_y, high_y]),
                                zaxis = dict(range=[high_z, low_z]),
                                xaxis_title="Longitude",
                                yaxis_title="Latitude",
                                zaxis_title="Depth",
                                aspectratio=dict(x=lon_lat_rel, y=1, z=0.5),
                                camera = dict(eye=dict(x=eye_x, y=eye_y, z=eye_z),
                                              center=dict(x=center_x, y=center_y, z=center_z)
                                             )
                                )
                     )

    return fig



app.run(debug=True,jupyter_height=1200) #jupyter_width,jupyter_height

In [78]:
ship_distances = re.compile(r"ship_\d_distance")
ship_distances = list(filter(ship_distances.match, df.columns))

noise_df[ship_distances].min()

ship_0_distance    1.536057
ship_1_distance    0.007016
dtype: float64

Minimum ship 1 distance is the retrieval/deployment boat, let's check without it:

In [79]:
noise_df.loc[noise_df["mmsi_1"] != 1,"ship_1_distance"].min()

4.24546087666734

In [80]:
noise_df.loc[noise_df["ship_0_distance"] == noise_df["ship_0_distance"].min()]

,time,m_altimeter_status,m_digifin_status,m_fin,m_is_ballast_pump_moving,m_is_battpos_moving,m_fin_diff,is_fin_moving,m_lat,m_lon,...,ship_1_distance,jussarö_wind_speed,jussarö_wind_direction,jussarö_gust_speed,jussarö_potential_wind,russarö_wind_speed,russarö_wind_direction,russarö_gust_speed,russarö_potential_wind,surface_depth
5692,2023-11-09 16:16:55.266000128,1.0,0.0,-0.034738,1.0,0.0,0.0,0.0,59.808338,23.288662,...,NaN,1.507633,31.92367,2.069211,1.507633,4.261064,173.53174,5.476587,3.822642,0


## Testing and Development

In [ ]:
min_distance_per_mmsi = threats_df.groupby("mmsi")["distance_from_glider"].min()
min_distance_per_mmsi

In [ ]:
len(min_distance_per_mmsi[min_distance_per_mmsi < 5])

In [ ]:
len(np.unique(df.time.dt.ceil(freq='min')))
df["min_freq_time"] = df.time.dt.ceil(freq='min')

In [ ]:
df.loc[df["longitude_1"].notna(), ["time", "name_1"]]

In [ ]:
df.loc[df["name_4"] == "NATO WARSHIP 511", ["time", "latitude_4", "longitude_4"]][100:150]

In [ ]:
fig = go.Figure()

trace_coordinate_list = [{"name":"Koskelo", "marker_symbol":"cross",
                          "x":"m_lon", "y":"m_lat", "z":"m_depth"},
                         {"name":"Uivelo", "marker_symbol":"cross", 
                          "x":"uivelo_m_lon", "y":"uivelo_m_lat", "z":"uivelo_m_depth"},
                         {"name":"Deployment", "marker_symbol":"diamond", 
                          "x":"lon", "y":"lat", "z":"surface_depth"},
                         {"name":"AUGUSTA", "marker_symbol":"diamond", 
                          "x":"longitude_0", "y":"latitude_0", "z":"surface_depth"},
                         {"name":"URSUS", "marker_symbol":"diamond", 
                         "x":"longitude_1", "y":"latitude_1", "z":"surface_depth"}]

for coords in trace_coordinate_list:
    fig.add_trace(go.Scatter3d(x=df[coords['x']], y=df[coords['y']], z=df[coords['z']], 
                           mode='lines', name=coords['name']))


for coords in trace_coordinate_list:
    for time in df["min_freq_time"].unique():
        fig.add_trace(
            go.Scatter3d(
                visible=False,
                mode="markers",
                #marker=dict(color="red", size=10),
                marker_size=8,
                marker_symbol=coords['marker_symbol'],
                name=coords['name'] + " at this time",
                x=df.loc[df['min_freq_time'] == time, coords['x']], 
                y=df.loc[df['min_freq_time'] == time, coords['y']], 
                z=df.loc[df['min_freq_time'] == time, coords['z']]))

# Create and add slider
steps = []
for time_idx,time in enumerate(df["min_freq_time"].unique()):
    step = dict(
        method="update",
        label= str(time),
        args=[{"visible": [False] * len(fig.data)}],
    )
    for path_idx,coords in enumerate(trace_coordinate_list):
        step["args"][0]["visible"][time_idx+len(df["min_freq_time"].unique())*path_idx] = True  # Toggle i'th trace to "visible"
        step["args"][0]["visible"][path_idx] = True  # Toggle whole path trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders,
                  scene=dict(zaxis = {'autorange': 'reversed'},
                             xaxis_title="Longitude",
                             yaxis_title="Latitude",
                             zaxis_title="Depth",
                             aspectratio=dict(x=1, y=2, z=0.95)))

fig.show()

In [ ]:
fig.write_html("locations_time_slider.html")

In [6]:
import re

latitudes = re.compile(".*lat.*")
latitudes = list(filter(latitudes.match, df.columns))

longitudes = re.compile(".*lon.*")
longitudes = list(filter(longitudes.match, df.columns))

max_lat = np.nanmax(df[latitudes])
min_lat = np.nanmin(df[latitudes])

max_lon = np.nanmax(df[longitudes])
min_lon = np.nanmin(df[longitudes])

max_depth = np.nanmax(df[["m_depth", "uivelo_m_depth"]])

In [7]:
max_lat - min_lat, max_lon - min_lon

(0.1038860022912047, 0.16245711313511535)

In [14]:
mean_lat_radians = np.radians((max_lat + min_lat)/2)
lon_lat_rel = np.cos(mean_lat_radians)

In [15]:
lon_lat_rel

0.5023987499917753

In [22]:
1/lon_lat_rel

1.9904508122609201

In [29]:
depth_lat_rel = 1/(1000*6371*np.pi/180)

In [24]:
mean_lat_radians

1.0444254914701459

In [28]:
1000*6371

6371000

In [30]:
depth_lat_rel

8.993216059187306e-06

In [9]:
noise_df = df.copy()

In [40]:
from dash import Dash, dcc, html, Input, Output
# import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    # html.H4('Iris samples filtered by petal width'),
    dcc.Graph(id="3d-scatter-plot-x-graph"),
    html.P("Longitude"),
    dcc.RangeSlider(
        id='3d-scatter-plot-x-range-slider',
        min=round(min_lon-0.01,2), max=round(max_lon+0.01,2), step=0.01,
        #marks={0: '0', 2.5: '2.5'},
        value=[round(min_lon-0.01,2), round(max_lon+0.01,2)]
    ),
    html.P("Latitude"),
    dcc.RangeSlider(
        id='3d-scatter-plot-y-range-slider',
        min=round(min_lat-0.01,2), max=round(max_lat+0.01,2), step=0.005,
        #marks={0: '0', 2.5: '2.5'},
        value=[round(min_lat-0.01,2), round(max_lat+0.01,2)]
    ),
    html.P("Depth"),
    dcc.RangeSlider(
        id='3d-scatter-plot-z-range-slider',
        min=0, max=np.ceil(max_depth), step=1,
        #marks={0: '0', 5: '5'},
        value=[0, np.ceil(max_depth)]
    ),
])


@app.callback(
    Output("3d-scatter-plot-x-graph", "figure"),
    [Input("3d-scatter-plot-x-range-slider", "value"),
    Input("3d-scatter-plot-y-range-slider", "value"),
    Input("3d-scatter-plot-z-range-slider", "value")])

def update_chart(slider_x, slider_y, slider_z):
    # df = px.data.iris()  # replace with your own data source
    low_x, high_x = slider_x
    low_y, high_y = slider_y
    low_z, high_z = slider_z
    # mask = (df.petal_width > low_x) & (df.petal_width < high_x) & (df.sepal_length > low_y) & (df.sepal_length < high_y) & (df.sepal_width > low_z) & (df.sepal_width < high_z)

    fig = go.Figure()

    trace_coordinate_list = [{"name":"Koskelo", "marker_symbol":"cross",
                            "x":"m_lon", "y":"m_lat", "z":"m_depth"},
                            {"name":"Uivelo", "marker_symbol":"cross", 
                            "x":"uivelo_m_lon", "y":"uivelo_m_lat", "z":"uivelo_m_depth"},
                            {"name":"Deployment", "marker_symbol":"diamond", 
                            "x":"lon", "y":"lat", "z":"surface_depth"},
                            {"name":"AUGUSTA", "marker_symbol":"diamond", 
                            "x":"longitude_0", "y":"latitude_0", "z":"surface_depth"},
                            {"name":"URSUS", "marker_symbol":"diamond", 
                            "x":"longitude_1", "y":"latitude_1", "z":"surface_depth"}]

    for coords in trace_coordinate_list:
        fig.add_trace(go.Scatter3d(x=df[coords['x']], y=df[coords['y']], z=df[coords['z']], 
                            mode='lines', name=coords['name']))


    for coords in trace_coordinate_list:
        for time in df["min_freq_time"].unique():
            fig.add_trace(
                go.Scatter3d(
                    visible=False,
                    mode="markers",
                    #marker=dict(color="red", size=10),
                    marker_size=8,
                    marker_symbol=coords['marker_symbol'],
                    name=coords['name'] + " at this time",
                    x=df.loc[df['min_freq_time'] == time, coords['x']], 
                    y=df.loc[df['min_freq_time'] == time, coords['y']], 
                    z=df.loc[df['min_freq_time'] == time, coords['z']]))

    # Create and add slider
    steps = []
    for time_idx,time in enumerate(df["min_freq_time"].unique()):
        step = dict(
            method="update",
            label= str(time),
            args=[{"visible": [False] * len(fig.data)}],
        )
        for path_idx,coords in enumerate(trace_coordinate_list):
            step["args"][0]["visible"][time_idx+len(df["min_freq_time"].unique())*path_idx] = True  # Toggle i'th trace to "visible"
            step["args"][0]["visible"][path_idx] = True  # Toggle whole path trace to "visible"
        steps.append(step)

    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Time: "},
        pad={"t": 50},
        steps=steps
    )]

    fig.update_layout(sliders=sliders,
                    scene=dict(xaxis = dict(range=[low_x, high_x]),
                                yaxis = dict(range=[low_y, high_y]),
                                zaxis = dict(range=[high_z, low_z]),
                                xaxis_title="Longitude",
                                yaxis_title="Latitude",
                                zaxis_title="Depth",
                                aspectratio=dict(x=lon_lat_rel, y=1, z=0.5))) # z=depth_lat_rel

    return fig



# app.run_server(debug=True, use_reloader=False)
app.run(debug=True) #jupyter_width,jupyter_height

In [10]:
df.index

RangeIndex(start=0, stop=6096, step=1)

In [14]:
df.loc[500, "m_lat"]

59.81918355238

In [14]:
from dash import Dash, dcc, html, Input, Output
# import plotly.express as px

app = Dash(__name__)

app.layout = html.Div([
    # html.H4('Iris samples filtered by petal width'),
    dcc.Graph(id="3d-scatter-plot-x-graph"),
    html.P("Time"),
    dcc.RangeSlider(
        id='3d-scatter-plot-t-range-slider',
        updatemode='drag',
        min=0, max=len(df)-1, step=1,
        marks={idx:'' for idx in range(len(df))}, # df['time'][idx]
        tooltip={"placement": "bottom", "always_visible": True},
        value=[0]
    ),
    html.P("Longitude"),
    dcc.RangeSlider(
        id='3d-scatter-plot-x-range-slider',
        min=round(min_lon-0.01,2), max=round(max_lon+0.01,2), step=0.01,
        #marks={0: '0', 2.5: '2.5'},
        value=[round(min_lon-0.01,2), round(max_lon+0.01,2)]
    ),
    html.P("Latitude"),
    dcc.RangeSlider(
        id='3d-scatter-plot-y-range-slider',
        min=round(min_lat-0.01,2), max=round(max_lat+0.01,2), step=0.005,
        #marks={0: '0', 2.5: '2.5'},
        value=[round(min_lat-0.01,2), round(max_lat+0.01,2)]
    ),
    html.P("Depth"),
    dcc.RangeSlider(
        id='3d-scatter-plot-z-range-slider',
        min=0, max=np.ceil(max_depth), step=1,
        #marks={0: '0', 5: '5'},
        value=[0, np.ceil(max_depth)]
    ),
])


@app.callback(
    Output("3d-scatter-plot-x-graph", "figure"),
    [Input("3d-scatter-plot-t-range-slider", "value"),
    Input("3d-scatter-plot-x-range-slider", "value"),
    Input("3d-scatter-plot-y-range-slider", "value"),
    Input("3d-scatter-plot-z-range-slider", "value")])

def update_trajectory_chart(slider_t, slider_x, slider_y, slider_z):
    # df = px.data.iris()  # replace with your own data source
    location_t = slider_t
    low_x, high_x = slider_x
    low_y, high_y = slider_y
    low_z, high_z = slider_z
    # mask = (df.petal_width > low_x) & (df.petal_width < high_x) & (df.sepal_length > low_y) & (df.sepal_length < high_y) & (df.sepal_width > low_z) & (df.sepal_width < high_z)

    fig = go.Figure()

    trace_coordinate_list = [{"name":"Koskelo", "marker_symbol":"cross",
                            "x":"m_lon", "y":"m_lat", "z":"m_depth"},
                            {"name":"Uivelo", "marker_symbol":"cross", 
                            "x":"uivelo_m_lon", "y":"uivelo_m_lat", "z":"uivelo_m_depth"},
                            {"name":"Deployment", "marker_symbol":"diamond", 
                            "x":"lon", "y":"lat", "z":"surface_depth"},
                            {"name":"AUGUSTA", "marker_symbol":"diamond", 
                            "x":"longitude_0", "y":"latitude_0", "z":"surface_depth"},
                            {"name":"URSUS", "marker_symbol":"diamond", 
                            "x":"longitude_1", "y":"latitude_1", "z":"surface_depth"}]

    for coords in trace_coordinate_list:
        fig.add_trace(go.Scatter3d(x=df[coords['x']], y=df[coords['y']], z=df[coords['z']], 
                            mode='lines', name=coords['name']))


    for coords in trace_coordinate_list:
        fig.add_trace(
            go.Scatter3d(
                mode="markers",
                #marker=dict(color="red", size=10),
                marker_size=8,
                marker_symbol=coords['marker_symbol'],
                name=coords['name'] + " at this time",
                x=df.loc[location_t, coords['x']],
                y=df.loc[location_t, coords['y']],
                z=df.loc[location_t, coords['z']]))

    fig.update_layout(title_text=str(df.loc[location_t[0], "time"]),
                      scene=dict(xaxis = dict(range=[low_x, high_x]),
                                yaxis = dict(range=[low_y, high_y]),
                                zaxis = dict(range=[high_z, low_z]),
                                xaxis_title="Longitude",
                                yaxis_title="Latitude",
                                zaxis_title="Depth",
                                aspectratio=dict(x=1, y=2, z=0.95)))

    return fig



# app.run_server(debug=True, use_reloader=False)
app.run(debug=True) #jupyter_width,jupyter_height

In [ ]:
import holoviews as hv
from holoviews import opts

hv.extension('plotly')

<img src='data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAABHNCSVQICAgIfAhkiAAAAAlwSFlz
AAAB+wAAAfsBxc2miwAAABl0RVh0U29mdHdhcmUAd3d3Lmlua3NjYXBlLm9yZ5vuPBoAAA6zSURB
VHic7ZtpeFRVmsf/5966taWqUlUJ2UioBBJiIBAwCZtog9IOgjqACsogKtqirT2ttt069nQ/zDzt
tI4+CrJIREFaFgWhBXpUNhHZQoKBkIUASchWla1S+3ar7r1nPkDaCAnZKoQP/D7mnPOe9/xy76n3
nFSAW9ziFoPFNED2LLK5wcyBDObkb8ZkxuaoSYlI6ZcOKq1eWFdedqNzGHQBk9RMEwFAASkk0Xw3
ETacDNi2vtvc7L0ROdw0AjoSotQVkKSvHQz/wRO1lScGModBFbDMaNRN1A4tUBCS3lk7BWhQkgpD
lG4852/+7DWr1R3uHAZVQDsbh6ZPN7CyxUrCzJMRouusj0ipRwD2uKm0Zn5d2dFwzX1TCGhnmdGo
G62Nna+isiUqhkzuKrkQaJlPEv5mFl2fvGg2t/VnzkEV8F5ioioOEWkLG86fvbpthynjdhXYZziQ
x1hC9J2NFyi8vCTt91Fh04KGip0AaG9zuCk2wQCVyoNU3Hjezee9bq92duzzTmxsRJoy+jEZZZYo
GTKJ6SJngdJqAfRzpze0+jHreUtPc7gpBLQnIYK6BYp/uGhw9YK688eu7v95ysgshcg9qSLMo3JC
4jqLKQFBgdKDPoQ+Pltb8dUyQLpeDjeVgI6EgLIQFT5tEl3rn2losHVsexbZ3EyT9wE1uGdkIPcy
BGxn8QUq1QrA5nqW5i2tLqvrrM9NK6AdkVIvL9E9bZL/oyfMVd/jqvc8LylzRBKDJSzIExwhQzuL
QYGQj4rHfFTc8mUdu3E7yoLtbTe9gI4EqVgVkug2i5+uXGo919ixbRog+3fTbQ8qJe4ZOYNfMoTI
OoshUNosgO60AisX15aeI2PSIp5KiFLI9ubb1vV3Qb2ltwLakUCDAkWX7/nHKRmmGIl9VgYsUhJm
2NXjKYADtM1ygne9QQDIXlk49FBstMKx66D1v4+XuQr7vqTe0VcBHQlRWiOCbmmSYe2SqtL6q5rJ
zsTb7lKx3FKOYC4DoqyS/B5bvLPxvD9Qtf6saxYLQGJErmDOdOMr/zo96km1nElr8bmPOBwI9COv
HnFPRIwmkSOv9kcAS4heRsidOkpeWBgZM+UBrTFAXNYL5Vf2ii9c1trNzpYdaoVil3WIc+wdk+gQ
noie3ecCcxt9ITcLAPWt/laGEO/9U6PmzZkenTtsSMQ8uYywJVW+grCstAvCIaAdArAsIWkRDDs/
KzLm2YcjY1Lv0UdW73HabE9n6V66cxSzfEmuJssTpKGVp+0vHq73FwL46eOjpMpbRAnNmJFrGJNu
Ukf9Yrz+3rghiumCKNXXWPhLYcjxGsIpoCMsIRoFITkW8AuyM8jC1+/QLx4bozCEJIq38+1rtpR6
V/yzb8eBlRb3fo5l783N0CWolAzJHaVNzkrTzlEp2bQ2q3TC5gn6wpnoQAmwSiGh2GitnTmVMc5O
UyfKWUKCIsU7+fZDKwqdT6DDpvkzAX4/+AMFjk0tDp5GRXLpQ2MUmhgDp5gxQT8+Y7hyPsMi8uxF
71H0oebujHALECjFKaW9Lm68n18wXp2kVzIcABytD5iXFzg+WVXkegpAsOOYziqo0OkK76GyquC3
ltZAzMhhqlSNmmWTE5T6e3IN05ITFLM4GdN0vtZ3ob8Jh1NAKXFbm5PtLU/eqTSlGjkNAJjdgn/N
aedXa0tdi7+t9G0FIF49rtMSEgAs1kDLkTPO7ebm4IUWeyh1bKomXqlgMG6kJmHcSM0clYLJ8XtR
1GTnbV3F6I5wCGikAb402npp1h1s7LQUZZSMIfALFOuL3UUrfnS8+rez7v9qcold5tilgHbO1fjK
9ubb17u9oshxzMiUBKXWqJNxd+fqb0tLVs4lILFnK71H0Ind7uiPgACVcFJlrb0tV6DzxqqTIhUM
CwDf1/rrVhTa33/3pGPxJYdQ2l2cbgVcQSosdx8uqnDtbGjh9SlDVSMNWhlnilfqZk42Th2ZpLpf
xrHec5e815zrr0dfBZSwzkZfqsv+1FS1KUknUwPARVvItfKUY+cn57yP7qv07UE3p8B2uhUwLk09
e0SCOrK+hbdYHYLjRIl71wWzv9jpEoeOHhGRrJAzyEyNiJuUqX0g2sBN5kGK6y2Blp5M3lsB9Qh4
y2Ja6x6+i0ucmKgwMATwhSjdUu49tKrQ/pvN5d53ml2CGwCmJipmKjgmyuaXzNeL2a0AkQ01Th5j
2DktO3Jyk8f9vcOBQHV94OK+fPumJmvQHxJoWkaKWq9Vs+yUsbq0zGT1I4RgeH2b5wef7+c7bl8F
eKgoHVVZa8ZPEORzR6sT1BzDUAD/d9F78e2Tzv99v8D+fLVTqAKAsbGamKey1Mt9Ann4eH3gTXTz
idWtAJ8PQWOk7NzSeQn/OTHDuEikVF1R4z8BQCy+6D1aWRfY0tTGG2OM8rRoPaeIj5ZHzJxszElN
VM8K8JS5WOfv8mzRnQAKoEhmt8gyPM4lU9SmBK1MCQBnW4KONT86v1hZ1PbwSXPw4JWussVjtH9Y
NCoiL9UoH/6PSu8jFrfY2t36erQHXLIEakMi1SydmzB31h3GGXFDFNPaK8Rme9B79Ixrd0WN+1ij
NRQ/doRmuFLBkHSTOm5GruG+pFjFdAmorG4IXH1Qua6ASniclfFtDYt+oUjKipPrCQB7QBQ2lrgP
fFzm+9XWUtcqJ3/5vDLDpJ79XHZk3u8nGZ42qlj1+ydtbxysCezrydp6ugmipNJ7WBPB5tydY0jP
HaVNzs3QzeE4ZpTbI+ZbnSFPbVOw9vsfnVvqWnirPyCNGD08IlqtYkh2hjZ5dErEQzoNm+6ykyOt
Lt5/PQEuSRRKo22VkydK+vvS1XEKlhCJAnsqvcVvH7f/ZU2R67eXbMEGAMiIV5oWZWiWvz5Fv2xG
sjqNJQRvn3Rs2lji/lNP19VjAQDgD7FHhujZB9OGqYxRkZxixgRDVlqS6uEOFaJUVu0rPFzctrnF
JqijImVp8dEKVWyUXDk92zAuMZ6bFwpBU1HrOw6AdhQgUooChb0+ItMbWJitSo5Ws3IAOGEOtL53
0vHZih9sC4vtofZ7Qu6523V/fmGcds1TY3V36pUsBwAbSlxnVh2xLfAD/IAIMDf7XYIkNmXfpp2l
18rkAJAy9HKFaIr/qULkeQQKy9zf1JgDB2uaeFNGijo5QsUyacNUUTOnGO42xSnv4oOwpDi1zYkc
efUc3I5Gk6PhyTuVKaOGyLUAYPGIoY9Pu/atL/L92+4q9wbflRJ2Trpm/jPjdBtfnqB/dIThcl8A
KG7hbRuKnb8qsQsVvVlTrwQAQMUlf3kwJI24Z4JhPMtcfng5GcH49GsrxJpGvvHIaeem2ma+KSjQ
lIwUdYyCY8j4dE1KzijNnIP2llF2wcXNnsoapw9XxsgYAl6k+KzUXbi2yP3KR2ecf6z3BFsBICdW
nvnIaG3eHybqX7vbpEqUMT+9OL4Qpe8VON7dXuFd39v19FoAABRVePbGGuXTszO0P7tu6lghUonE
llRdrhArLvmKdh9u29jcFiRRkfLUxBiFNiqSU9icoZQHo5mYBI1MBgBH6wMNb+U7Pnw337H4gi1Y
ciWs+uks3Z9fztUvfzxTm9Ne8XXkvQLHNytOOZeiD4e0PgkAIAYCYknKUNUDSXEKzdWNpnil7r4p
xqkjTarZMtk/K8TQ6Qve78qqvXurGwIJqcOUKfUWHsm8KGvxSP68YudXq4pcj39X49uOK2X142O0
Tz5/u/7TVybqH0rSya6ZBwD21/gubbrgWdDgEOx9W

In [ ]:
path = hv.Path3D(df[["m_lat", "m_lon", "m_depth"]]).opts(invert_zaxis=True)

path

In [ ]:
hv.Scatter3D(df, kdims=["m_lat", "m_lon", "m_depth"])

In [ ]:
hv.HoloMap({time: hv.Scatter3D(df.loc[df["time"] == time], kdims=["m_lat", "m_lon", "m_depth"], group='Group', label='Label') for time in df["time"]}, 'Time')

Animation longer than the max_frames limit 500;
skipping rendering to avoid unexpected lengthy computations.
If desired, the limit can be increased using:
hv.output(max_frames=<insert number>)

:HoloMap   [Time]
   :Scatter3D   [m_lat,m_lon,m_depth]   (time,m_ballast_pumped,m_battpos,m_fin,m_ballast_pumped_diff,m_battpos_diff,m_fin_diff,is_ballast_moving,is_battpos_moving,is_fin_moving,uivelo_m_lat,uivelo_m_lon,uivelo_m_depth,glider_distance,mmsi_0,longitude_0,latitude_0,locUpdatetime_0,draught_0,mmsi_1,longitude_1,latitude_1,locUpdatetime_1,draught_1,ship_0_distance,ship_1_distance,lat,lon,boat_distance,surface_depth)

In [ ]:
# Holoviews 3D plot of paths

paths_list = [df[["m_lat", "m_lon", "m_depth"]], df[["uivelo_m_lat", "uivelo_m_lon", "uivelo_m_depth"]], 
              df[["lat", "lon", "surface_depth"]], 
              df[["latitude_0", "longitude_0", "surface_depth"]], df[["latitude_1", "longitude_1", "surface_depth"]]]

paths = [{('x', 'y', 'z'): path, 'index': i} for i,path in enumerate(paths_list)]

hv.Path3D(paths, vdims='index').opts(color='index',invert_zaxis=True)

In [ ]:
# Holoviews 3D plot of paths

path_labels = ["Koskelo", "Uivelo", "Deployment", "AUGUSTA", "URSUS"]

koskelo_path = hv.Path3D(df[["m_lat", "m_lon", "m_depth"]], label="Koskelo")
uivelo_path = hv.Path3D(df[["uivelo_m_lat", "uivelo_m_lon", "uivelo_m_depth"]], label="Uivelo")
deployment_path = hv.Path3D(df[["lat", "lon", "surface_depth"]], label="Deployment")
augusta_path = hv.Path3D(df[["latitude_0", "longitude_0", "surface_depth"]], label="AUGUSTA")
ursus_path = hv.Path3D(df[["latitude_1", "longitude_1", "surface_depth"]], label="URSUS")

paths = (koskelo_path*uivelo_path*deployment_path*augusta_path*ursus_path).opts(invert_zaxis=True,show_legend=True,width=1500,height=800)
paths

In [ ]:
hv.save(paths, "2023-11-09_paths.html")

In [ ]:
# Per-minute animation in html of Koskelo's position with play button and slider using plotly express

fig = px.scatter_3d(df, x='m_lat', y='m_lon', z='m_depth', animation_frame="min_freq_time", 
                    range_x=[min(df['m_lat']),max(df['m_lat'])], 
                    range_y=[min(df['m_lon']),max(df['m_lon'])],
                    range_z=[min(df['m_depth']),max(df['m_depth'])])
fig.add_trace(go.Scatter3d(x=df['m_lat'], y=df['m_lon'], z=df['m_depth'], mode='lines', name="Koskelo"))

fig.update_layout(
    scene={
        'zaxis': {'autorange': 'reversed'}
    }
)
fig.write_html("test.html")
fig.show()

In [ ]:
# Generate curve data
t = np.linspace(-1, 1, 100)
x = t + t ** 2
y = t - t ** 2
xm = np.min(x) - 1.5
xM = np.max(x) + 1.5
ym = np.min(y) - 1.5
yM = np.max(y) + 1.5
N = 50
s = np.linspace(-1, 1, N)
xx = s + s ** 2
yy = s - s ** 2


# Create figure
fig = go.Figure(
    data=[go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue")),
          go.Scatter(x=x, y=y,
                     mode="lines",
                     line=dict(width=2, color="blue"))],
    layout=go.Layout(
        xaxis=dict(range=[xm, xM], autorange=False, zeroline=False),
        yaxis=dict(range=[ym, yM], autorange=False, zeroline=False),
        title_text="Kinematic Generation of a Planar Curve", hovermode="closest",
        updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None])])]),
    frames=[go.Frame(
        data=[go.Scatter(
            x=[xx[k]],
            y=[yy[k]],
            mode="markers",
            marker=dict(color="red", size=10))])

        for k in range(N)]
)

fig.show()

In [ ]:
# Per-minute animation in html of Koskelo's position with play button

x=df['m_lat']
y=df['m_lon']
z=df['m_depth']

fig = go.Figure(
    data=[go.Scatter3d(x=[], y=[], z=[],
                     mode="markers",marker=dict(color="red", size=10))])
    
fig.update_layout(
        
         scene = dict(
        
        xaxis=dict(range=[np.nanmin(x), np.nanmax(x)], autorange=False),
        yaxis=dict(range=[np.nanmin(y), np.nanmax(y)], autorange=False),
        zaxis=dict(range=[np.nanmin(z), np.nanmax(z)], autorange=False),
        )),


frames=[go.Frame(
        data=[go.Scatter3d(
            x=df.loc[df['min_freq_time'] == time, 'm_lat'], 
            y=df.loc[df['min_freq_time'] == time, 'm_lon'], 
            z=df.loc[df['min_freq_time'] == time, 'm_depth'])],
            traces=[0],
            name=str(time))

        for time in df["min_freq_time"].unique()]

fig.update(frames=frames),


fig.update_layout(updatemenus=[dict(type="buttons",
                          buttons=[dict(label="Play",
                                        method="animate",
                                        args=[None, dict(frame=dict(redraw=True,fromcurrent=True, mode='immediate'))      ])])])

fig.add_trace(go.Scatter3d(x=df['m_lat'], y=df['m_lon'], z=df['m_depth'], mode='lines', name="Koskelo", line=dict(width=2, color="blue")))


fig.show()

In [ ]:
# Per-minute frames in html of Koskelo's position with slider

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=df['m_lat'], y=df['m_lon'], z=df['m_depth'], mode='lines', name="Koskelo", line=dict(width=2, color="blue")))

for time in df["min_freq_time"].unique():
    fig.add_trace(
        go.Scatter3d(
            visible=False,
            mode="markers",
            marker=dict(color="red", size=10),
            name=str(time),
            x=df.loc[df['min_freq_time'] == time, 'm_lat'], 
            y=df.loc[df['min_freq_time'] == time, 'm_lon'], 
            z=df.loc[df['min_freq_time'] == time, 'm_depth']))
    

fig.data[0].visible = True

# Create and add slider
steps = []
for i,time in enumerate(df["min_freq_time"].unique()):
    step = dict(
        method="update",
        label= str(time),
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][0] = True  # Toggle whole path trace to "visible"
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(sliders=sliders,
                  scene={'zaxis': {'autorange': 'reversed'}})

fig.show()

In [27]:
import re

latitudes = re.compile(".*lat.*")
latitudes = list(filter(latitudes.match, df.columns))

longitudes = re.compile(".*lon.*")
longitudes = list(filter(longitudes.match, df.columns))

In [31]:
max_lat = np.nanmax(df[latitudes])
min_lat = np.nanmin(df[latitudes])

max_lon = np.nanmax(df[longitudes])
min_lon = np.nanmin(df[longitudes])

In [35]:
lat_diff = max_lat - min_lat
lat_diff

0.1038860022912047

In [36]:
lon_diff = max_lon - min_lon
lon_diff

0.16245711313511535

In [ ]:
ships_df = pd.DataFrame()

start = datetime.strptime("2023-11-18 00:00","%Y-%m-%d %H:%M").timestamp()*1000
end = datetime.strptime("2023-11-21 00:00","%Y-%m-%d %H:%M").timestamp()*1000

# Neither of these databases covers the whole mission and they overlap
for db in ["AIS.sqlite", "AIS2.sqlite"]:
    conn = sqlite3.connect(db)

    query = ("SELECT longitude,latitude,locAPICallTimestamp FROM locations "
            f"WHERE locAPICallTimestamp > '{start}' "
            f"AND locAPICallTimestamp < '{end}' ")

    ships_df = pd.concat([ships_df, pd.read_sql_query(query, conn)], ignore_index=True)
    conn.close()

# ships_df = ships_df.loc[:,~ships_df.columns.duplicated()] # If mmsi column duplicates from missing values in meta, drop the extra column

ships_df.drop_duplicates(inplace=True) # Some API calls don't have new location info
                 
ships_df["locAPICallTimestamp"] = pd.to_datetime(ships_df["locAPICallTimestamp"], unit="ms")

ships_df.sort_values(by=["locAPICallTimestamp"], inplace=True)

ships_df.reset_index(drop=True, inplace=True)

ships_df

In [43]:
len(ships_df["locAPICallTimestamp"].unique())

426

In [54]:
len(ships_df.loc[(ships_df["longitude"] > 23.22-1.5) & (ships_df["longitude"] < 23.22+1.5) & (ships_df["latitude"] > 59.4) & (ships_df["latitude"] < 59.8), "locAPICallTimestamp"].unique())

388

In [56]:
interesting_times = ships_df["locAPICallTimestamp"].unique()[~ships_df["locAPICallTimestamp"].unique().isin(ships_df.loc[(ships_df["longitude"] > 23.22-1.5) & 
                                                                                                                         (ships_df["longitude"] < 23.22+1.5) & 
                                                                                                                         (ships_df["latitude"] > 59.4) & 
                                                                                                                         (ships_df["latitude"] < 59.8), "locAPICallTimestamp"].unique())]
interesting_times

<DatetimeArray>
['2023-11-17 22:40:02.616000', '2023-11-17 22:50:02.670000',
 '2023-11-18 03:40:02.192000', '2023-11-18 03:50:02.529000',
 '2023-11-18 07:40:02.532000', '2023-11-18 07:50:02.618000',
 '2023-11-18 10:40:02.827000', '2023-11-18 10:50:02.198000',
 '2023-11-18 20:10:02.898000', '2023-11-18 20:20:02.358000',
 '2023-11-18 22:10:02.714000', '2023-11-18 22:20:02.521000',
 '2023-11-19 04:40:02.841000', '2023-11-19 04:50:02.099000',
 '2023-11-19 08:10:02.731000', '2023-11-19 08:20:02.954000',
 '2023-11-19 12:40:02.408000', '2023-11-19 12:50:02.565000',
 '2023-11-19 13:40:02.419000', '2023-11-19 13:50:02.508000',
 '2023-11-19 14:40:02.904000', '2023-11-19 14:50:02.037000',
 '2023-11-19 20:40:01.964000', '2023-11-19 20:50:02.003000',
 '2023-11-19 21:20:02.288000', '2023-11-19 22:40:02.097000',
 '2023-11-19 22:50:02.613000', '2023-11-20 01:20:02.077000',
 '2023-11-20 04:10:02.030000', '2023-11-20 04:20:02.430000',
 '2023-11-20 06:10:02.277000', '2023-11-20 06:20:02.608000',
 '2023-1

In [37]:
interesting_times.round(freq='H').unique()

<DatetimeArray>
['2023-11-09 09:00:00', '2023-11-09 10:00:00', '2023-11-09 11:00:00',
 '2023-11-09 15:00:00', '2023-11-09 17:00:00', '2023-11-09 18:00:00',
 '2023-11-10 06:00:00', '2023-11-10 11:00:00', '2023-11-10 12:00:00',
 '2023-11-10 13:00:00', '2023-11-10 18:00:00', '2023-11-10 20:00:00',
 '2023-11-10 23:00:00', '2023-11-11 00:00:00', '2023-11-11 01:00:00',
 '2023-11-11 02:00:00', '2023-11-11 03:00:00', '2023-11-11 07:00:00',
 '2023-11-11 11:00:00', '2023-11-11 12:00:00', '2023-11-11 13:00:00',
 '2023-11-12 03:00:00', '2023-11-12 04:00:00', '2023-11-12 10:00:00',
 '2023-11-12 17:00:00', '2023-11-12 18:00:00', '2023-11-12 19:00:00',
 '2023-11-13 01:00:00', '2023-11-13 03:00:00', '2023-11-13 06:00:00',
 '2023-11-13 07:00:00', '2023-11-13 08:00:00', '2023-11-13 13:00:00',
 '2023-11-13 14:00:00', '2023-11-14 02:00:00', '2023-11-14 04:00:00',
 '2023-11-14 05:00:00', '2023-11-14 07:00:00', '2023-11-14 09:00:00',
 '2023-11-14 12:00:00']
Length: 40, dtype: datetime64[ns]

In [57]:
with open(f'maybe_quiet.txt', 'a') as file:
        for time in interesting_times:
            file.write(str(time) + "\n")
        file.close()

In [11]:
ships_df = pd.DataFrame()

start = datetime.strptime("2023-11-09 08:50 +0000","%Y-%m-%d %H:%M %z").timestamp()*1000
end = datetime.strptime("2023-11-11 00:00 +0000","%Y-%m-%d %H:%M %z").timestamp()*1000

# Neither of these databases covers the whole mission and they overlap
for db in ["AIS.sqlite"]:
    conn = sqlite3.connect(f"./Noise Data/{db}")

    query = ("SELECT name,locAPICallTimestamp FROM threats "
            f"WHERE locAPICallTimestamp > '{start}' "
            f"AND locAPICallTimestamp < '{end}' "
            "AND glider_name = 'Koskelo' "
            "AND distance_from_glider <= 10 ")
            #"GROUP BY locAPICallTimestamp")

    ships_df = pd.concat([ships_df, pd.read_sql_query(query, conn)], ignore_index=True)
    conn.close()
                 
ships_df["locAPICallTimestamp"] = pd.to_datetime(ships_df["locAPICallTimestamp"], unit="ms")

ships_df.sort_values(by=["locAPICallTimestamp"], inplace=True)

ships_df.reset_index(drop=True, inplace=True)

ships_df

,name,locAPICallTimestamp
0,AUGUSTA,2023-11-09 12:30:01.841
1,AUGUSTA,2023-11-09 12:40:02.711
2,AUGUSTA,2023-11-09 12:50:02.252
3,AUGUSTA,2023-11-09 13:00:02.693
4,AUGUSTA,2023-11-09 13:10:02.040
5,AUGUSTA,2023-11-09 13:20:02.723
6,AUGUSTA,2023-11-09 13:30:02.176
7,AUGUSTA,2023-11-09 13:30:02.176
8,URSUS,2023-11-09 15:40:02.727
9,URSUS,2023-11-09 15:50:02.419
